In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import duckdb
from pathlib import Path
import re
import seaborn as sns

In [ ]:
ROOT = Path.cwd().resolve()

DATA = (ROOT / "data").resolve()

PROV_SVC_DATA = (DATA / "prov_svc").resolve()

PROV_DATA = (DATA / "prov").resolve()

# Reading the Data in Using Pandas (Later Skipped / Commented Out)

# Provider Service Data

In [ ]:
# prov_svc_2021 = pd.read_csv(PROV_SVC_DATA / "MUP_PHY_R25_P05_V20_D21_Prov_Svc.csv")

In [ ]:
# prov_svc_2021["Year"] = 2021

In [ ]:
# prov_svc_2021.info()

In [ ]:
# prov_svc_2022 = pd.read_csv(PROV_SVC_DATA / "MUP_PHY_R25_P05_V20_D22_Prov_Svc.csv")

In [ ]:
# prov_svc_2022["Year"] = 2022

In [ ]:
# prov_svc_2022.info()

In [ ]:
# prov_svc_2023 = pd.read_csv(PROV_SVC_DATA / "MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv")

In [ ]:
# prov_svc_2023["Year"] = 2023

In [ ]:
# prov_svc_2023.info()

In [ ]:
# prov_svc_all = pd.concat([prov_svc_2021, prov_svc_2022, prov_svc_2023], axis= 0).reset_index(drop=True)

In [ ]:
# prov_svc_all.columns

In [ ]:
# prov_svc_all.info()

In [ ]:
# prov_svc_all.info()

# Provider Data

In [ ]:
# prov_2021 = pd.read_csv(PROV_DATA / "MUP_PHY_R25_P07_V20_D21_Prov.csv")

In [ ]:
# prov_2021["Year"] = 2021

In [ ]:
# [c for c in prov_2021.columns if "Cancer" in c]

In [ ]:
# [c for c in prov_2021.columns if "Bene_CC_PH" in c]

In [ ]:
# prov_2021.info()

In [ ]:
# prov_2022 = pd.read_csv(PROV_DATA / "MUP_PHY_R25_P07_V20_D22_Prov.csv")

In [ ]:
# prov_2022["Year"] = 2022

In [ ]:
# [c for c in prov_2022.columns if "Cancer" in c]

In [ ]:
# [c for c in prov_2022.columns if "Bene_CC_PH" in c]

In [ ]:
# prov_2022.info()

In [ ]:
# prov_2023 = pd.read_csv(PROV_DATA / "MUP_PHY_R25_P05_V20_D23_Prov.csv")

In [ ]:
# prov_2023["Year"] = 2023

In [ ]:
# [c for c in prov_2023.columns if "Cancer" in c]

In [ ]:
# [c for c in prov_2023.columns if "Bene_CC_PH" in c]

In [ ]:
# prov_2023.info()

In [ ]:
# prov_all = pd.concat([prov_2021, prov_2022, prov_2023], axis=0).reset_index(drop=True)

In [ ]:
# prov_all.columns

In [ ]:
# [c for c in prov_all.columns if "Cancer" in c]

In [ ]:
# prov_all.info()

In [ ]:
# prov_svc_all.head(10)

In [ ]:
# prov_svc_all["Rndrng_Prvdr_Ent_Cd"].value_counts()

In [ ]:
# onco_prvdr_type = [s for s in prov_svc_all["Rndrng_Prvdr_Type"].value_counts().index.to_list() if "onco" in s.lower()]
# onco_prvdr_type

In [ ]:
# counts_df = prov_svc_all["Rndrng_Prvdr_Type"].value_counts().reset_index().rename(columns={"count": "Count"})

# onco_counts = counts_df[counts_df["Rndrng_Prvdr_Type"].isin(onco_prvdr_type)]

# onco_counts

In [ ]:
# prov_svc_all_cdI_typeOnco = prov_svc_all[(prov_svc_all["Rndrng_Prvdr_Ent_Cd"]=="I") & prov_svc_all["Rndrng_Prvdr_Type"].isin(onco_prvdr_type)].reset_index()

In [ ]:
# prov_svc_all_cdI_typeOnco.info()

In [ ]:
# prov_svc_all_cdI_typeOnco["Rndrng_Prvdr_Type"].value_counts()

Note:

Above steps were too memory-intensive, so I switched to DuckDB altogether for loading and inspecting the each individual dataset. 

Above, I only read in Provider Service and Provider Datasets. 

Using DuckDB, I will create a database that creates views for Provider Service, Provider, RCBS Taxonomy, and NPPES datasets. I'll sanity checks and quality control to ensure the the intended data is loaded correctly, the views are filtered for target analysis, appropriate views are joined and the final metadata is saved as a table, a Pandas dataframe and a parquet file. 

# SWITCH TO DUCKDB FOR EFFICIENCY (USE THE FOR THE REST OF THE NOTEBOOK)

In [ ]:
con = duckdb.connect(database=DATA / "medicare.duckdb")

# ----------------------------
# Step 1: Provider-service union (3 years)
# ----------------------------

con.execute(
    f"""
CREATE OR REPLACE VIEW prov_svc_all AS
SELECT *, 2021 AS Year
FROM read_csv_auto('{(PROV_SVC_DATA / "MUP_PHY_R25_P05_V20_D21_Prov_Svc.csv").as_posix()}')
UNION ALL
SELECT *, 2022 AS Year 
FROM read_csv_auto('{(PROV_SVC_DATA / "MUP_PHY_R25_P05_V20_D22_Prov_Svc.csv").as_posix()}')
UNION ALL
SELECT *, 2023 As Year
FROM read_csv_auto('{(PROV_SVC_DATA / "MUP_PHY_R25_P05_V20_D23_Prov_Svc.csv").as_posix()}');
"""
)

Visit [Medicare Physician & Other Practitioners - by Provider and Service Data Dictionary](https://data.cms.gov/resources/medicare-physician-other-practitioners-by-provider-and-service-data-dictionary).

In [ ]:
# Oncology provider types: contains "onco"

con.execute("""
CREATE OR REPLACE VIEW prov_svc_onco AS
SELECT *
FROM prov_svc_all
WHERE Rndrng_Prvdr_Ent_Cd = 'I'
  AND Rndrng_Prvdr_Type IN (
    'Hematology-Oncology',
    'Medical Oncology',
    'Radiation Oncology',
    'Gynecological Oncology',
    'Surgical Oncology'
  );
""")



In [ ]:
con.execute("""
CREATE OR REPLACE VIEW prov_svc_onco_core AS
SELECT *,
  CASE
    WHEN Rndrng_Prvdr_Type IN ('Hematology-Oncology', 'Medical Oncology', 'Radiation Oncology')
      THEN 1 ELSE 0
  END AS is_core_scope
FROM prov_svc_onco;
""")

In [ ]:
con.sql(
    """
SELECT Rndrng_Prvdr_Type, is_core_scope, COUNT(*) AS n
FROM prov_svc_onco_core
GROUP BY 1,2
ORDER BY n DESC;
"""
)

In [ ]:
con.execute(
    f"""
CREATE OR REPLACE VIEW prov_all AS
SELECT *, 2021 AS Year 
FROM read_csv_auto('{(PROV_DATA / "MUP_PHY_R25_P07_V20_D21_Prov.csv").as_posix()}')
UNION ALL
SELECT *, 2022 AS Year
FROM read_csv_auto('{(PROV_DATA / "MUP_PHY_R25_P07_V20_D22_Prov.csv").as_posix()}')
UNION ALL 
SELECT *, 2023 AS Year
FROM read_csv_auto('{(PROV_DATA / "MUP_PHY_R25_P05_V20_D23_Prov.csv").as_posix()}');
"""
)

Visit: [Medicare Physician & Other Practitioners - by Provider Data Dictionary](https://data.cms.gov/resources/medicare-physician-other-practitioners-by-provider-data-dictionary)

In [ ]:
con.execute(
    """
CREATE OR REPLACE VIEW prov_svc_onco_core_prov_all AS
SELECT
  ps.*,
  p.* EXCLUDE (Rndrng_NPI, Year)
FROM prov_svc_onco_core AS ps
LEFT JOIN prov_all AS p
  ON ps.Rndrng_NPI = p.Rndrng_NPI
 AND ps.Year = p.Year;
""")

Let's confirm we didn’t accidentally create duplicate provider rows in `prov_all`

In [ ]:
con.sql(
    """
SELECT Year, COUNT(*) AS n_rows, COUNT(DISTINCT Rndrng_NPI) AS n_npi
FROM prov_all
GROUP BY 1
ORDER BY 1;
"""
)

Let's check match rate after the left join

In [ ]:
con.sql(
    """
SELECT COUNT(*) AS n_rows,
       AVG(CASE WHEN Tot_Benes IS NULL THEN 1 ELSE 0 END) AS pct_missing_provider_summary
FROM prov_svc_onco_core_prov_all
"""
)

In [ ]:
RBCS_DATA = (DATA / "rbcs").resolve()

In [ ]:
con.execute(
    f"""
CREATE OR REPLACE VIEW rbcs AS
SELECT *
FROM read_csv_auto('{(RBCS_DATA / "RBCS_Taxonomy_RY2025.csv").as_posix()}');
""")

***CRITICAL NOTE: At the end of "EDA 6. Specialty comparisons (our 5 oncology types)" I discovered that the original `rbcs` dataset has duplicates that led to fatal downstream duplication.*** 

The code below fixes the `rbcs` dataset by removing duplicates and generating a clean dataset called `rbcs_dedup`. 

We will rebuild the `prov_svc_onco_core_prov_all_rbcs` dataset with the clean `rbcs_dedup` and run the entire notebook from start to finish to ensure the duplication issue is completely resolved.

In [ ]:
con.execute(
    """
CREATE OR REPLACE VIEW rbcs_dedup AS
SELECT *
FROM (
  SELECT
    r.*,
    ROW_NUMBER() OVER (
      PARTITION BY HCPCS_Cd
      ORDER BY
        RBCS_Latest_Assignment DESC,
        RBCS_Analysis_Start_Dt DESC NULLS LAST,
        RBCS_Analysis_End_Dt DESC NULLS LAST,
        CASE WHEN RBCS_FamNumb = '000' THEN 1 ELSE 0 END
    ) AS rn
  FROM rbcs r
)
WHERE rn = 1;
"""
)

After deduping, confirm RBCS is now unique on HCPCS:

In [ ]:
con.sql(
    """
SELECT
  COUNT(*) AS rbcs_dedup_rows,
  COUNT(DISTINCT HCPCS_Cd) AS distinct_hcpcs
FROM rbcs_dedup;
"""
)

Visit: [Restructured Berenson-Eggers Type of Service (BETOS) Classification System (RBCS) Data Dictionary](https://data.cms.gov/sites/default/files/2022-05/RBCS%20Mapping%20Data%20Dictionary.pdf)

***CRITICAL NOTE: DO NOT UNCOMMENT THE CODE BELOW. THIS IS KEPT FOR "REFERENCE ONLY". UNCOMMENTING IT MIGHT CAUSE DUPLICATION ISSUES.***

In [ ]:
# THIS CODE IS DEPRICATED! DO NOT UNCOMMENT THIS!

# con.execute(
#     """
# CREATE OR REPLACE VIEW prov_svc_onco_core_prov_all_rbcs AS
# SELECT p.*, r.* EXCLUDE(HCPCS_Cd)
# FROM prov_svc_onco_core_prov_all AS p
# LEFT JOIN rbcs AS r
#   ON p.HCPCS_Cd = r.HCPCS_Cd;
# """)

Let's reconstruct the `prov_svc_onco_core_prov_all_rbcs` using the clean `rbcs_dedup`.

In [ ]:
con.execute(
    """
CREATE OR REPLACE VIEW prov_svc_onco_core_prov_all_rbcs AS
SELECT p.*, r.* EXCLUDE(HCPCS_Cd)
FROM prov_svc_onco_core_prov_all AS p
LEFT JOIN rbcs_dedup AS r
  ON p.HCPCS_Cd = r.HCPCS_Cd;
"""
)

**Must-run check after the rebuild:** The specific offender should stop doubling:

In [ ]:
con.sql(
    """
SELECT
  'pre'  AS stage, COUNT(*) n_rows, SUM(Tot_Srvcs) sum_srvcs
FROM prov_svc_onco_core_prov_all
WHERE Rndrng_NPI=1912087271 AND Year=2021
UNION ALL
SELECT
  'post' AS stage, COUNT(*) n_rows, SUM(Tot_Srvcs) sum_srvcs
FROM prov_svc_onco_core_prov_all_rbcs
WHERE Rndrng_NPI=1912087271 AND Year=2021;
"""
)

Let's check whether the join is matching codes as expected

In [ ]:
con.sql(
    """
SELECT COUNT(*) AS n_rows,
AVG(CASE WHEN RBCS_Id IS NULL THEN 1 ELSE 0 END) AS pct_missing_rbcs
FROM prov_svc_onco_core_prov_all_rbcs;
"""
)

In [ ]:
NPPES_DATA = (DATA / "nppes").resolve()

In [ ]:
con.execute(
    f"""
CREATE OR REPLACE VIEW nppes_raw AS 
SELECT * 
FROM read_csv_auto('{(NPPES_DATA / "npidata_pfile_20050523-20260111.csv").as_posix()}');
"""
)

In [ ]:
con.sql(
    """
SELECT *
FROM nppes_raw
LIMIT 4
"""
)

In [ ]:
con.sql("DESCRIBE SELECT * FROM nppes_raw").df().head(60)

Let's create a slim NPPES view

In [ ]:
con.execute(
    """
CREATE OR REPLACE VIEW nppes AS 
SELECT NPI,
       "Entity Type Code" AS entity_type_code,
       "Provider Enumeration Date" AS enumeration_date
FROM nppes_raw
WHERE Entity_Type_Code = 1;
"""
)

In [ ]:
con.sql(
    """
SELECT *
FROM nppes
LIMIT 4
"""
)

In [ ]:
con.sql(
    """
SELECT * 
FROM nppes
"""
).df().columns

#### Let's do a quick sanity check. 

In [ ]:
con.sql(
    """
SELECT
  COUNT(*) AS n_rows,
  MIN(enumeration_date) AS min_date,
  MAX(enumeration_date) AS max_date
FROM nppes;
"""
)

#### Let's left join the `prov_svc_onco_core_prov_all_rbcs` table with the `nppes` table on `Rndrng_NPI` on the left side and `NPI` on the right side.

In [ ]:
con.execute(
    """
CREATE OR REPLACE VIEW prov_svc_onco_core_prov_all_rbcs_nppes AS
SELECT
    t.*,
    n.enumeration_date,
    CASE
        WHEN n.enumeration_date IS NULL
        THEN NULL
        ELSE (t.Year - EXTRACT(year FROM n.enumeration_date))::INTEGER
    END AS years_since_enumeration
FROM prov_svc_onco_core_prov_all_rbcs AS t
LEFT JOIN nppes AS n
    ON CAST(t.Rndrng_NPI AS BIGINT) = n.NPI;
"""
)

In [ ]:
con.sql(
    """
SELECT
  COUNT(*) AS n_rows,
  AVG(CASE WHEN enumeration_date IS NULL THEN 1 ELSE 0 END) AS pct_missing_enumeration_date,
  MIN(years_since_enumeration) AS min_years,
  MAX(years_since_enumeration) AS max_years
FROM prov_svc_onco_core_prov_all_rbcs_nppes;
"""
)

In [ ]:
con.execute(
    """
CREATE OR REPLACE TABLE prov_svc_onco_core_prov_all_rbcs_nppes_tbl AS
SELECT * FROM prov_svc_onco_core_prov_all_rbcs_nppes;
"""
)

In [ ]:
OUT_PARQUET = (DATA / "prov_svc_onco_core_prov_all_rbcs_nppes.parquet").as_posix()

con.execute(
    f"""
COPY prov_svc_onco_core_prov_all_rbcs_nppes_tbl
TO '{OUT_PARQUET}'
(FORMAT PARQUET);
"""
)

#### Reading the master dataset in using Pandas:

In [ ]:
import pandas as pd
df = pd.read_parquet(DATA / "prov_svc_onco_core_prov_all_rbcs_nppes.parquet")
df.shape
df.head()

In [ ]:
desc = con.sql(
    """
DESCRIBE 
    SELECT *
    FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl;
"""
).df()

desc.head(50)

#### Table 1: provider_year_features (small, stable, best for EDA + clustering)

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE provider_year_features AS
SELECT
  Rndrng_NPI,
  Year,

  -- provider identity (should be constant within NPI-Year)
  MAX(Rndrng_Prvdr_Type) AS provider_type,
  MAX(is_core_scope) AS is_core_scope,
  MAX(Rndrng_Prvdr_State_Abrvtn) AS state,
  MAX(Rndrng_Prvdr_Zip5) AS zip5,
  MAX(Rndrng_Prvdr_RUCA) AS ruca,
  MAX(Rndrng_Prvdr_RUCA_Desc) AS ruca_desc,

  -- provider-year totals (from provider summary)
  MAX(Tot_HCPCS_Cds) AS tot_hcpcs_cds,
  MAX(Tot_Benes_1) AS tot_benes,
  MAX(Tot_Srvcs_1) AS tot_srvcs,
  MAX(Tot_Sbmtd_Chrg) AS tot_sbmtd_chrg,
  MAX(Tot_Mdcr_Alowd_Amt) AS tot_mdcr_allowed_amt,
  MAX(Tot_Mdcr_Pymt_Amt) AS tot_mdcr_payment_amt,
  MAX(Tot_Mdcr_Stdzd_Amt) AS tot_mdcr_stdzd_amt,

  -- derived unit costs at provider-year level
  MAX(Tot_Mdcr_Stdzd_Amt) / NULLIF(MAX(Tot_Srvcs_1), 0) AS stdzd_amt_per_service,
  MAX(Tot_Mdcr_Pymt_Amt) / NULLIF(MAX(Tot_Srvcs_1), 0) AS payment_amt_per_service,
  MAX(Tot_Mdcr_Alowd_Amt) / NULLIF(MAX(Tot_Srvcs_1), 0) AS allowed_amt_per_service,

  -- drug vs medical totals (optional but useful)
  MAX(Drug_Tot_Srvcs) AS drug_tot_srvcs,
  MAX(Drug_Mdcr_Stdzd_Amt) AS drug_mdcr_stdzd_amt,
  MAX(Med_Tot_Srvcs) AS med_tot_srvcs,
  MAX(Med_Mdcr_Stdzd_Amt) AS med_mdcr_stdzd_amt,

  -- case-mix proxies
  MAX(Bene_Avg_Risk_Scre) AS bene_avg_risk_score,

  -- demographics counts (optional)
  MAX(Bene_Avg_Age) AS bene_avg_age,
  MAX(Bene_Feml_Cnt) AS bene_female_cnt,
  MAX(Bene_Male_Cnt) AS bene_male_cnt,
  MAX(Bene_Dual_Cnt) AS bene_dual_cnt,
  MAX(Bene_Ndual_Cnt) AS bene_nondual_cnt,

  -- chronic condition percentages (keep as features, even if they are BIGINT)
  MAX(Bene_CC_PH_Cancer6_V2_Pct) AS pct_cancer6,
  MAX(Bene_CC_PH_Diabetes_V2_Pct) AS pct_diabetes,
  MAX(Bene_CC_PH_CKD_V2_Pct) AS pct_ckd,
  MAX(Bene_CC_PH_COPD_V2_Pct) AS pct_copd,
  MAX(Bene_CC_PH_Hypertension_V2_Pct) AS pct_htn,

  -- experience proxy (from NPPES join)
  MAX(years_since_enumeration) AS years_since_enumeration

FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
GROUP BY 1, 2;
""")

In [ ]:
OUT = (DATA / "provider_year_features.parquet").as_posix()
con.execute(f"COPY provider_year_features TO '{OUT}' (FORMAT PARQUET);")

#### Table 2: provider_service_features (modeling table for regression benchmark)

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE provider_service_features AS
SELECT
  Rndrng_NPI,
  Year,
  MAX(Rndrng_Prvdr_Type) AS provider_type,
  MAX(is_core_scope) AS is_core_scope,

  -- service grouping
  RBCS_FamNumb,
  MAX(RBCS_Family_Desc) AS rbcs_family_desc,
  MAX(RBCS_Cat) AS rbcs_cat,
  MAX(RBCS_Cat_Subcat) AS rbcs_cat_subcat,

  -- context
  Place_Of_Srvc,
  MAX(Rndrng_Prvdr_State_Abrvtn) AS state,
  MAX(Rndrng_Prvdr_Zip5) AS zip5,
  MAX(Rndrng_Prvdr_RUCA) AS ruca,

  -- volume and exposure
  SUM(Tot_Srvcs) AS services,
  SUM(Tot_Benes) AS benes,
  SUM(Tot_Bene_Day_Srvcs) AS bene_day_services,

  -- weighted average “price” targets (per service)
  SUM(Avg_Mdcr_Stdzd_Amt * Tot_Srvcs) / NULLIF(SUM(Tot_Srvcs), 0) AS stdzd_amt_per_service,
  SUM(Avg_Mdcr_Alowd_Amt * Tot_Srvcs) / NULLIF(SUM(Tot_Srvcs), 0) AS allowed_amt_per_service,
  SUM(Avg_Mdcr_Pymt_Amt * Tot_Srvcs) / NULLIF(SUM(Tot_Srvcs), 0) AS payment_amt_per_service,
  SUM(Avg_Sbmtd_Chrg * Tot_Srvcs) / NULLIF(SUM(Tot_Srvcs), 0) AS submitted_charge_per_service,

  -- provider-level features (repeated across service rows, so take MAX)
  MAX(Bene_Avg_Risk_Scre) AS bene_avg_risk_score,
  MAX(years_since_enumeration) AS years_since_enumeration,

  -- a few case-mix features (add more later if we want)
  MAX(Bene_CC_PH_Cancer6_V2_Pct) AS pct_cancer6,
  MAX(Bene_CC_PH_Diabetes_V2_Pct) AS pct_diabetes,
  MAX(Bene_CC_PH_CKD_V2_Pct) AS pct_ckd,
  MAX(Bene_CC_PH_COPD_V2_Pct) AS pct_copd,
  MAX(Bene_CC_PH_Hypertension_V2_Pct) AS pct_htn

FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
WHERE RBCS_FamNumb IS NOT NULL  -- keep only rows mapped to RBCS
GROUP BY 1,2,5,9;
""")

In [ ]:
OUT = (DATA / "provider_service_features.parquet").as_posix()
con.execute(f"COPY provider_service_features TO '{OUT}' (FORMAT PARQUET);")

## QC or sanity checks on the newly created dataframes

### QC A. Row counts and uniqueness at the intended grain

1) provider_year_features should be unique on (Rndrng_NPI, Year)

In [ ]:
con.sql("""
SELECT
  COUNT(*) AS n_rows,
  COUNT(DISTINCT (Rndrng_NPI, Year)) AS n_unique_keys,
  COUNT(*) - COUNT(DISTINCT (Rndrng_NPI, Year)) AS n_duplicate_keys
FROM provider_year_features;
""").df()

2) provider_service_features should be unique on (Rndrng_NPI, Year, RBCS_FamNumb, Place_Of_Srvc)

In [ ]:
con.sql("""
SELECT
  COUNT(*) AS n_rows,
  COUNT(DISTINCT (Rndrng_NPI, Year, RBCS_FamNumb, Place_Of_Srvc)) AS n_unique_keys,
  COUNT(*) - COUNT(DISTINCT (Rndrng_NPI, Year, RBCS_FamNumb, Place_Of_Srvc)) AS n_duplicate_keys
FROM provider_service_features;
""").df()

### QC B. Missingness and “must-have” fields

3) Provider-year: check missing core fields

In [ ]:
con.sql("""
SELECT
  AVG(CASE WHEN tot_benes IS NULL THEN 1 ELSE 0 END) AS pct_missing_tot_benes,
  AVG(CASE WHEN tot_srvcs IS NULL THEN 1 ELSE 0 END) AS pct_missing_tot_srvcs,
  AVG(CASE WHEN tot_mdcr_stdzd_amt IS NULL THEN 1 ELSE 0 END) AS pct_missing_tot_stdzd_amt,
  AVG(CASE WHEN bene_avg_risk_score IS NULL THEN 1 ELSE 0 END) AS pct_missing_risk,
  AVG(CASE WHEN years_since_enumeration IS NULL THEN 1 ELSE 0 END) AS pct_missing_years_since_enum
FROM provider_year_features;
""").df()

4) Provider-service: check mapping and targets

In [ ]:
con.sql("""
SELECT
  AVG(CASE WHEN RBCS_FamNumb IS NULL THEN 1 ELSE 0 END) AS pct_missing_family,
  AVG(CASE WHEN services IS NULL OR services = 0 THEN 1 ELSE 0 END) AS pct_zero_services,
  AVG(CASE WHEN stdzd_amt_per_service IS NULL THEN 1 ELSE 0 END) AS pct_missing_stdzd,
  AVG(CASE WHEN allowed_amt_per_service IS NULL THEN 1 ELSE 0 END) AS pct_missing_allowed,
  AVG(CASE WHEN payment_amt_per_service IS NULL THEN 1 ELSE 0 END) AS pct_missing_payment
FROM provider_service_features;
""").df()

### QC C. Validate the weighted-average math behaves

5) Spot-check that the “per service” amounts are in sane ranges

Run percentiles to detect garbage (like negative, huge, or all zeros):

In [ ]:
con.sql("""
SELECT
  quantile_cont(stdzd_amt_per_service, 0.01) AS p01_stdzd,
  quantile_cont(stdzd_amt_per_service, 0.50) AS p50_stdzd,
  quantile_cont(stdzd_amt_per_service, 0.99) AS p99_stdzd,
  MIN(stdzd_amt_per_service) AS min_stdzd,
  MAX(stdzd_amt_per_service) AS max_stdzd
FROM provider_service_features
WHERE services >= 11;
""").df()

In [ ]:
con.sql("""
SELECT
  quantile_cont(allowed_amt_per_service, 0.01) AS p01_stdzd,
  quantile_cont(allowed_amt_per_service, 0.50) AS p50_stdzd,
  quantile_cont(allowed_amt_per_service, 0.99) AS p99_stdzd,
  MIN(allowed_amt_per_service) AS min_stdzd,
  MAX(allowed_amt_per_service) AS max_stdzd
FROM provider_service_features
WHERE services >= 11;
""").df()

In [ ]:
con.sql("""
SELECT
  quantile_cont(payment_amt_per_service, 0.01) AS p01_stdzd,
  quantile_cont(payment_amt_per_service, 0.50) AS p50_stdzd,
  quantile_cont(payment_amt_per_service, 0.99) AS p99_stdzd,
  MIN(payment_amt_per_service) AS min_stdzd,
  MAX(payment_amt_per_service) AS max_stdzd
FROM provider_service_features
WHERE services >= 11;
""").df()

Check how often the target is exactly zero (because the p01 is 0.0 across all three targets).


In [ ]:
con.sql("""
SELECT
  AVG(CASE WHEN stdzd_amt_per_service = 0 THEN 1 ELSE 0 END) AS pct_zero_stdzd,
  AVG(CASE WHEN allowed_amt_per_service = 0 THEN 1 ELSE 0 END) AS pct_zero_allowed,
  AVG(CASE WHEN payment_amt_per_service = 0 THEN 1 ELSE 0 END) AS pct_zero_payment
FROM provider_service_features;
""").df()

### QC D. Reconciliation checks (to ensure we didn’t accidentally double-count)

6) Provider-year totals vs sum of provider-service “reconstructed totals”

In [ ]:
con.sql("""
WITH svc_recon AS (
  SELECT
    Rndrng_NPI,
    Year,
    SUM(stdzd_amt_per_service * services) AS recon_stdzd_amt,
    SUM(services) AS recon_services
  FROM provider_service_features
  GROUP BY 1,2
),
joined AS (
  SELECT
    y.Rndrng_NPI,
    y.Year,
    y.tot_mdcr_stdzd_amt,
    y.tot_srvcs,
    s.recon_stdzd_amt,
    s.recon_services,
    (s.recon_stdzd_amt / NULLIF(y.tot_mdcr_stdzd_amt, 0)) AS ratio_stdzd_amt,
    (s.recon_services / NULLIF(y.tot_srvcs, 0)) AS ratio_services
  FROM provider_year_features y
  LEFT JOIN svc_recon s
    ON y.Rndrng_NPI = s.Rndrng_NPI
   AND y.Year = s.Year
)
SELECT
  COUNT(*) AS n,
  AVG(CASE WHEN ratio_stdzd_amt BETWEEN 0.5 AND 1.5 THEN 1 ELSE 0 END) AS pct_ratio_stdzd_in_0_5_to_1_5,
  quantile_cont(ratio_stdzd_amt, 0.50) AS median_ratio_stdzd,
  quantile_cont(ratio_services, 0.50) AS median_ratio_services
FROM joined
WHERE tot_mdcr_stdzd_amt > 0 AND tot_srvcs > 0;
""").df()

### QC E. Quick sanity distributions by year and specialty

In [ ]:
con.sql("""
SELECT
  Year,
  provider_type,
  COUNT(*) AS n_providers,
  AVG(stdzd_amt_per_service) AS avg_stdzd_per_service,
  AVG(bene_avg_risk_score) AS avg_risk
FROM provider_year_features
GROUP BY 1,2
ORDER BY 1, n_providers DESC;
""").df()

## Creating the EDA dataset

In [ ]:
OUT = (DATA / "eda_dataset.parquet").as_posix()

con.execute("""
CREATE OR REPLACE TABLE eda_dataset AS
WITH base AS (
  SELECT
    -- keys
    ps.Rndrng_NPI,
    ps.Year,
    ps.RBCS_FamNumb,
    ps.Place_Of_Srvc,

    -- labels for readability (not for modeling)
    ps.rbcs_family_desc,
    ps.rbcs_cat,
    ps.rbcs_cat_subcat,

    -- provider context
    ps.provider_type,
    ps.is_core_scope,
    ps.state,
    ps.zip5,
    ps.ruca,

    -- derived RUCA bucket (simplifies plots and modeling later)
    CASE
      WHEN ps.ruca IS NULL THEN 'Unknown'
      WHEN ps.ruca <= 3 THEN 'Urban'
      WHEN ps.ruca <= 6 THEN 'Suburban'
      ELSE 'Rural'
    END AS ruca_bucket,

    -- exposure
    ps.services,
    ps.benes,
    ps.bene_day_services,

    -- observed “price” outcomes (per service)
    ps.stdzd_amt_per_service,
    ps.allowed_amt_per_service,
    ps.payment_amt_per_service,
    ps.submitted_charge_per_service,

    -- observed totals (useful for spend concentration plots)
    ps.stdzd_amt_per_service * ps.services AS stdzd_spend,
    ps.allowed_amt_per_service * ps.services AS allowed_spend,
    ps.payment_amt_per_service * ps.services AS payment_spend,
    ps.submitted_charge_per_service * ps.services AS submitted_spend,

    -- log transforms for EDA and later modeling
    ln(1 + ps.stdzd_amt_per_service) AS log_stdzd_amt_per_service,
    ln(1 + ps.services) AS log_services,
    ln(1 + ps.benes) AS log_benes,

    -- case-mix and experience
    ps.bene_avg_risk_score,
    ps.years_since_enumeration,

    -- condition percentage features (keep both raw and scaled)
    ps.pct_cancer6,
    ps.pct_diabetes,
    ps.pct_ckd,
    ps.pct_copd,
    ps.pct_htn,

    (CAST(ps.pct_cancer6 AS DOUBLE) / 100.0) AS p_cancer6,
    (CAST(ps.pct_diabetes AS DOUBLE) / 100.0) AS p_diabetes,
    (CAST(ps.pct_ckd AS DOUBLE) / 100.0) AS p_ckd,
    (CAST(ps.pct_copd AS DOUBLE) / 100.0) AS p_copd,
    (CAST(ps.pct_htn AS DOUBLE) / 100.0) AS p_htn,

    -- optional provider-year totals for context checks
    py.tot_benes,
    py.tot_srvcs,
    py.tot_mdcr_stdzd_amt,
    py.stdzd_amt_per_service AS provider_year_stdzd_amt_per_service

  FROM provider_service_features AS ps
  LEFT JOIN provider_year_features AS py
    ON ps.Rndrng_NPI = py.Rndrng_NPI
   AND ps.Year = py.Year

  WHERE ps.services >= 11
    AND ps.stdzd_amt_per_service IS NOT NULL
    AND ps.stdzd_amt_per_service >= 0
),
q AS (
  -- exact percentile (can swap to APPROX_QUANTILE for speed)
  SELECT quantile_cont(stdzd_amt_per_service, 0.99) AS q99
  FROM base
)
SELECT
  b.*,

  -- mirror pandas: is_top_1pct_stdzd_amt_per_service = stdzd_amt_per_service >= q99
  (b.stdzd_amt_per_service >= q.q99) AS is_top_1pct_stdzd_amt_per_service,

  -- mirror pd.cut(... right=False): left-inclusive, right-exclusive bins
  CASE
    WHEN b.services < 11 THEN '<11'
    WHEN b.services >= 11 AND b.services < 25 THEN '11-25'
    WHEN b.services >= 25 AND b.services < 50 THEN '25-50'
    WHEN b.services >= 50 AND b.services < 100 THEN '50-100'
    WHEN b.services >= 100 AND b.services < 500 THEN '100-500'
    ELSE '500+'
  END AS svc_bucket

FROM base b
CROSS JOIN q;
""")

con.execute(f"COPY eda_dataset TO '{OUT}' (FORMAT PARQUET);")

A sanity check that the top 1% flag is about right:

In [ ]:
con.sql(
    """
SELECT
  AVG(CASE WHEN is_top_1pct_stdzd_amt_per_service THEN 1 ELSE 0 END) AS pct_flagged
FROM eda_dataset;
"""
)

And verify bucket boundaries:

In [ ]:
con.sql(
    """
SELECT svc_bucket, MIN(services) AS min_s, MAX(services) AS max_s, COUNT(*) AS n
FROM eda_dataset
GROUP BY 1
ORDER BY 1;
"""
)

## EDA 0. Setup and data load

**Code**

- Load parquet
- Print shape, dtypes
- Show the grain: confirm unique key counts

**Questions answered**

- What is the unit of analysis?
- How big is the dataset after feasibility filters?

In [ ]:
eda_df = pd.read_parquet(DATA/"eda_dataset.parquet")

In [ ]:
eda_df.shape

In [ ]:
eda_df.dtypes

In [ ]:
eda_df.columns

`tot_benes`: How many distinct beneficiaries did this provider see in this year across all services and places of service

`benes`: How many distinct beneficiaries received at least one service in this specific family and POS from this provider in this year?

So, we cannot sum `benes` and expect to get `tot_benes` for a specific provider-year pair. Let's demonstrate this:

In [ ]:
eda_df["Rndrng_NPI"].sample(2)

In [ ]:
eda_df["Year"].sample(1)

In [ ]:
eda_df.query("Rndrng_NPI == 1629379565 and Year == 2022")[["benes","tot_benes","Year", "Rndrng_NPI","Place_Of_Srvc","rbcs_family_desc"]].head()


- `tot_benes` = 354 means:
    - Provider 1629379565 had 354 unique Medicare beneficiaries in 2022 across all services and places of service that feed into your provider-year table.

- Row-level `benes` means:
    - Within each provider-year + family + place of service slice, how many unique beneficiaries had at least one service in that slice.

So in our sample rows:
- Office E&M Established, POS=O: 51 unique beneficiaries had at least one such service.
- IMRT, POS=O: 245 unique beneficiaries had at least one IMRT service.
- Office E&M New, POS=O: 66 unique beneficiaries had at least one new-patient E&M service.
- Conventional Radiation Treatment, POS=O: 327 unique beneficiaries had at least one conventional radiation service.
- Telephone Services, POS=O: 23 unique beneficiaries had at least one telephone service.

And crucially:
Those beneficiary sets can overlap heavily (likely they do here).
That is why you can see big `benes` values across multiple families while `tot_benes` stays 354.

Same idea applies to `services` vs `tot_srvcs`. 
Let's quickly check:

Interpretation:
- share ≈ 1 means your row table and provider-year totals align (same universe, no duplication).
- share < 1 usually means your row table is a subset (filters).
- share > 1 is a red flag for duplication or a universe mismatch where row table includes more than provider-year table.


In [ ]:
con.sql(
    """
SELECT
  SUM(services) AS sum_services_rows,
  MAX(tot_srvcs) AS tot_srvcs_py,
  SUM(services) * 1.0 / NULLIF(MAX(tot_srvcs), 0) AS share
FROM eda_dataset
WHERE Rndrng_NPI = 1629379565 AND Year = 2022;
"""
)

**What these numbers say:**

- `tot_srvcs_py` = `11817` means: provider `1629379565` delivered `11,817` total services in `2022` in the provider-year summary table’s universe.
- `sum_services_rows` = `11410` means: when you sum services across all rows you kept in `eda_dataset` for that provider-year, you capture `11,410` services.

So you are missing:
- 11817 - 11410 = 407 services
- Share captured = 11410 / 11817 = 0.9656 (about 96.6%)

Interpretation:

This is a good sign.

- Share < 1 means our row-level table is not “more” than the provider-year total. So it is not behaving like a duplication problem.
- Share close to 1 means our row-level table covers most of the provider-year services, but not all. That is consistent with our filters and/or scope restrictions.

What could explain the missing 3.4% (407 services)?
Most likely one of these, and our pipeline already suggests them:

1. We filtered to core oncology specialties (is_core_scope)

- If `tot_srvcs` was computed for the provider-year regardless of specialty restrictions, our EDA table would drop some services.

2. Service-family mapping coverage

- Some HCPCS lines might not land in our included RBCS families or might fall into categories we excluded upstream.
- Even “No RBCS Family” does not guarantee we captured every HCPCS, depending on how our family tables were built.

3. Place-of-service scope differences

- If our EDA rows only include certain POS codes (even implicitly, based on what exists in the service table we used), we might miss some POS contributions that still count in provider-year totals.

4. Other feasibility filters

- We filtered out null or negative `stdzd_amt_per_service` (and we require `services >= 11` at the row grain). Those filters can remove some rows that still contribute to the provider-year totals.

**One quick follow-up diagnostic:**

In [ ]:
con.sql(
    """
SELECT
  Place_Of_Srvc,
  SUM(services) AS sum_services,
  MAX(tot_srvcs) AS tot_srvcs_py,
  SUM(services) * 1.0 / NULLIF(MAX(tot_srvcs), 0) AS share
FROM eda_dataset
WHERE Rndrng_NPI = 1629379565 AND Year = 2022
GROUP BY 1
ORDER BY sum_services DESC;
"""
)


In [ ]:
con.sql(
    """
SELECT
  rbcs_family_desc,
  SUM(services) AS sum_services
FROM eda_dataset
WHERE Rndrng_NPI = 1629379565 AND Year = 2022
GROUP BY 1
ORDER BY sum_services DESC
LIMIT 30;
"""
)

We did the “within eda_dataset” breakdown correctly. The reason you still cannot see the missing 407 there is simple:
- Our POS table is computed from `eda_dataset`, and `eda_dataset` already only contains rows that passed your EDA filters (`ps.services >= 11`, `stdzd_amt_per_service IS NOT NULL`, `stdzd_amt_per_service >= 0`, plus whatever is upstream in provider_service_features).
- So the 407 `services` are not in `eda_dataset` at all. That is why they do not show up under any POS or any family in our POS and family tables.

Let's run this on `provider_service_features` (the table we SELECT FROM to build `eda_dataset`):

- If `services_that_would_enter_eda_dataset` = `11410` and `services_in_provider_service_features` = 11817, then the 407 are being dropped by the EDA filters.
- If `services_in_provider_service_features` is already 11410 (or close), then the 407 never made it into `provider_service_features` in the first place. That points to an upstream scope restriction (specialty filtering, family mapping, etc.).


**Step 1. Confirm the loss happens at the EDA filtering step (most likely)**

In [ ]:
con.sql(
    """
WITH base AS (
  SELECT *
  FROM provider_service_features
  WHERE Rndrng_NPI = 1629379565 AND Year = 2022
)
SELECT
  SUM(services) AS services_in_provider_service_features,

  SUM(CASE
        WHEN services >= 11
         AND stdzd_amt_per_service IS NOT NULL
         AND stdzd_amt_per_service >= 0
        THEN services ELSE 0 END) AS services_that_would_enter_eda_dataset,

  SUM(services) - SUM(CASE
        WHEN services >= 11
         AND stdzd_amt_per_service IS NOT NULL
         AND stdzd_amt_per_service >= 0
        THEN services ELSE 0 END) AS services_dropped_by_eda_filters
FROM base;
"""
)


**Step 2. Attribute the dropped services to specific reasons**

If `dropped_services_lt_11` is about 407, we have our answer: they are low-volume slices that we intentionally excluded for reliability.


In [ ]:
con.sql(
    """
WITH base AS (
  SELECT *
  FROM provider_service_features
  WHERE Rndrng_NPI = 1629379565 AND Year = 2022
)
SELECT
  SUM(CASE WHEN services < 11 THEN services ELSE 0 END) AS dropped_services_lt_11,

  SUM(CASE WHEN services >= 11 AND stdzd_amt_per_service IS NULL THEN services ELSE 0 END) AS dropped_services_stdzd_null,

  SUM(CASE WHEN services >= 11 AND stdzd_amt_per_service < 0 THEN services ELSE 0 END) AS dropped_services_stdzd_neg,

  SUM(services) AS total_services_in_base
FROM base;
"""
)


**Step 3. Show the exact dropped rows (so it’s not a mystery)**

This is the “where did they go” table.

This will show us the specific families and POS where those services live, and which rule excluded them.


In [ ]:
con.sql(
    """
SELECT
  Place_Of_Srvc,
  rbcs_family_desc,
  RBCS_FamNumb,
  services,
  benes,
  stdzd_amt_per_service,
  allowed_amt_per_service,
  payment_amt_per_service,
  submitted_charge_per_service
FROM provider_service_features
WHERE Rndrng_NPI = 1629379565 AND Year = 2022
  AND (
       services < 11
    OR stdzd_amt_per_service IS NULL
    OR stdzd_amt_per_service < 0
  )
ORDER BY services DESC
LIMIT 100;
"""
)


**Step 4. If the loss is upstream, compare against the raw service table**

Since Step 1 showed `provider_service_features` already below 11817, then we want to compare raw totals:


In [ ]:
con.sql(
    """
SELECT
  SUM(Tot_Srvcs) AS raw_sum_tot_srvcs
FROM prov_svc_all_prov_all
WHERE Rndrng_NPI = 1629379565 AND Year = 2022;
"""
)


In [ ]:
con.sql(
    """
SELECT
  SUM(services) AS services_in_provider_service_features
FROM provider_service_features
WHERE Rndrng_NPI = 1629379565 AND Year = 2022;
"""
)


In [ ]:
con.sql("""
WITH
psf AS (
  SELECT SUM(services) AS svc
  FROM provider_service_features
  WHERE Rndrng_NPI=1629379565 AND Year=2022
),
eda AS (
  SELECT SUM(services) AS svc
  FROM eda_dataset
  WHERE Rndrng_NPI=1629379565 AND Year=2022
),
raw AS (
  SELECT SUM(Tot_Srvcs) AS svc
  FROM prov_svc_all_prov_all
  WHERE Rndrng_NPI=1629379565 AND Year=2022
),
py AS (
  SELECT MAX(tot_srvcs) AS svc
  FROM provider_year_features
  WHERE Rndrng_NPI=1629379565 AND Year=2022
)
SELECT
  psf.svc AS services_provider_service_features,
  eda.svc AS services_eda_dataset,
  raw.svc AS services_raw_table_sum,
  py.svc  AS tot_srvcs_provider_year_features
FROM psf
CROSS JOIN eda
CROSS JOIN raw
CROSS JOIN py;
""")

Perfect. This is exactly the outcome we were trying to isolate.

What this proves (very clearly)

For this provider-year (NPI 1629379565, 2022):
- Raw service-line table sum = 11410
- provider_service_features sum = 11410
- eda_dataset sum = 11410

So our service-line universe is internally consistent end-to-end.

But:
- `provider_year_features.tot_srvcs` = 11817

So the “missing 407 services” are not missing from our service-line tables at all. They are only “missing” relative to the provider-year total table, which is using a broader definition of total services than what ended up in `prov_svc_all_prov_all` (and therefore in `provider_service_features` and `eda_dataset`).

Numerically:
- 11817 − 11410 = 407 services
- That is about 3.44% of the provider-year total (407 / 11817).

Why this happens (most likely)

`provider_year_features.tot_srvcs` is probably calculated from a different upstream table or rule set, for example:
- includes non-oncology specialties, non-core-scope, or non-included HCPCS/RBCS mappings
- includes rows you filtered out earlier in the pipeline (not at EDA time, but earlier during “oncology-core” construction)
- includes POS buckets or claim types not present in our final service-line extract
- includes services outside our RBCS mapping universe (even though you’re keeping “No RBCS Family” inside the service-line table).

We want to identify the universe used to compute `provider_year_features.tot_srvcs`. The fastest diagnostic is:

A) Find the exact table used to build `provider_year_features`
Where did `provider_year_features` come from in our notebook? Search for its CREATE TABLE statement and check which source table it aggregates.

B) Quantify the gap by computing provider-year totals from the same base as eda_dataset
If you want totals that reconcile by construction, compute a `“tot_srvcs_from_psf”` (or from the same service-line base) and carry that alongside the existing `tot_srvcs`.



Let's check the unique row count per the our eda grain: Provider, Year, RBCS family, Place of service:

In [ ]:
eda_df[["Rndrng_NPI","Year","RBCS_FamNumb","Place_Of_Srvc"]].count()

Checking the number of uniqe values each variables has 

In [ ]:
eda_df.nunique().sort_values(ascending=False).head(20)

Let's prove what one row represents in our eda dataset by checking that the columns we claim define a row are unique.

In the EDA dataset, the grain is supposed to be:

1 row = (Rndrng_NPI, Year, RBCS_FamNumb, Place_Of_Srvc)

In [ ]:
key_cols = ["Rndrng_NPI","Year","RBCS_FamNumb","Place_Of_Srvc"]

n_rows = eda_df.shape[0]
n_unique = eda_df[key_cols].drop_duplicates().shape[0]
n_dupe = n_rows - n_unique

print("n_rows:", n_rows)
print("n_unique_keys:", n_unique)
print("n_duplicate_rows:", n_dupe)

**KEY NOTE: In EDA 8, we found that rows in the top 1% of `stdzd_amt_per_service` have a median service volume of 41. The median `stdzd_amt_per_service` among the top 1% is ~736.7, compared with ~70.3 for the bottom 99%.**

After this discovery, I decided to define tail flags and make them reusable. 

So let's create indicators once and use them everywhere:
- `is_top_1pct_stdzd_amt_per_service`
- and a service-volume (i.e., `services`) bucket: `svc_bucket`

In [ ]:
q99 = eda_df["stdzd_amt_per_service"].quantile(0.99)

eda_df = eda_df.assign(
    is_top_1pct_stdzd_amt_per_service = eda_df["stdzd_amt_per_service"] >= q99,
    svc_bucket = pd.cut(
        eda_df["services"],
        bins=[0, 11, 25, 50, 100, 500, np.inf],
        labels=["<11", "11-25", "25-50", "50-100", "100-500", "500+"],
        right=False
    )
)

In [ ]:
eda_df["is_top_1pct_stdzd_amt_per_service"].mean()

## EDA 1. Data quality and completeness

**Plots / tables**

1. Missingness table (percent missing per column)
2. Category sanity tables:
    - counts of `provider_type`
    - counts of `Place_Of_Srvc`
    - counts of `ruca_bucket`
    - top 20 `rbcs_family_desc` by rows

**Questions answered**

- Are there missing values that could bias modeling?
- Do we have unexpected categories or weird values?

In [ ]:
miss_cnt = eda_df.isna().sum().rename("n_missing")
miss_pct = (eda_df.isna().mean()*100).rename("pct_missing")

In [ ]:
miss=(
    pd.concat([miss_cnt,miss_pct],axis=1)
    .reset_index(names="column")
    .query("n_missing>0")
    .sort_values("n_missing", ascending=False))

In [ ]:
plt.figure(figsize=(10, max(4,0.5*len(miss))))

g = sns.barplot(data=miss, x="n_missing", y="column", orient="h", color="#000080")

max_missing = miss["n_missing"].max()

g.set(
    title="Missing values by column",
    xlabel="Number of missing values",
    ylabel=""
)

# 3) Annotate % missing
for i, (cnt, pct) in enumerate(zip(miss["n_missing"], miss["pct_missing"])):
    # Offset text slightly to the right of the bar
    g.text(cnt + (max_missing * 0.002), i, f"{pct:.1f}%", va="center", fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
eda_df["is_top_1pct_stdzd_amt_per_service"].value_counts()

In [ ]:
is_top_1pct_eda_df = eda_df.loc[eda_df["is_top_1pct_stdzd_amt_per_service"]==True, :]

miss_cnt_is_top_1pct = is_top_1pct_eda_df.isna().sum().rename("n_missing")
miss_pct_is_top_1pct = (is_top_1pct_eda_df.isna().mean()*100).rename("pct_missing")

miss_top_1pct=(
    pd.concat([miss_cnt_is_top_1pct,miss_pct_is_top_1pct],axis=1)
    .reset_index(names="column")
    .query("n_missing>0")
    .sort_values("n_missing", ascending=False))

plt.figure(figsize=(10, max(4,0.5*len(miss_top_1pct))))

g = sns.barplot(data=miss_top_1pct, x="n_missing", y="column", orient="h", color="#000080")

max_missing = miss_top_1pct["n_missing"].max()

g.set(
    title="Missing values by column (top 1%)",
    xlabel="Number of missing values",
    ylabel=""
)

# 3) Annotate % missing
for i, (cnt, pct) in enumerate(zip(miss_top_1pct["n_missing"], miss_top_1pct["pct_missing"])):
    # Offset text slightly to the right of the bar
    g.text(cnt + (max_missing * 0.002), i, f"{pct:.1f}%", va="center", fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
is_bottom_99pct_eda_df = eda_df.loc[eda_df["is_top_1pct_stdzd_amt_per_service"]==False, :]

miss_cnt_is_bottom_99pct = is_bottom_99pct_eda_df.isna().sum().rename("n_missing")
miss_pct_is_bottom_99pct = (is_bottom_99pct_eda_df.isna().mean()*100).rename("pct_missing")

miss_bottom_99pct=(
    pd.concat([miss_cnt_is_bottom_99pct,miss_pct_is_bottom_99pct],axis=1)
    .reset_index(names="column")
    .query("n_missing>0")
    .sort_values("n_missing", ascending=False))

plt.figure(figsize=(10, max(4,0.5*len(miss_bottom_99pct))))

g = sns.barplot(data=miss_bottom_99pct, x="n_missing", y="column", orient="h", color="#000080")

max_missing = miss_bottom_99pct["n_missing"].max()

g.set(
    title="Missing values by column (bottom 99%)",
    xlabel="Number of missing values",
    ylabel=""
)

# 3) Annotate % missing
for i, (cnt, pct) in enumerate(zip(miss_bottom_99pct["n_missing"], miss_bottom_99pct["pct_missing"])):
    # Offset text slightly to the right of the bar
    g.text(cnt + (max_missing * 0.002), i, f"{pct:.1f}%", va="center", fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
eda_df[["provider_type","Place_Of_Srvc","ruca_bucket","rbcs_family_desc"]].nunique()

In [ ]:
eda_df["provider_type"].unique()

In [ ]:
eda_df["Place_Of_Srvc"].unique()

In [ ]:
eda_df["ruca_bucket"].unique()

In [ ]:
eda_df["rbcs_family_desc"].unique()[0:20]

Important realization: `“No RBCS Family”` showing up in `rbcs_family_desc` is not automatically wrong, but it is worth verifying what it corresponds to. It might be a real label in the RBCS file, or it might indicate some mapping fallback.

Quick check: If it’s truly mapped, RBCS_FamNumb should still be present and consistent.

In [ ]:
con.sql(
    """
SELECT
  rbcs_family_desc,
  COUNT(*) AS n,
  MIN(RBCS_FamNumb) AS min_fam,
  MAX(RBCS_FamNumb) AS max_fam
FROM provider_service_features
WHERE rbcs_family_desc = 'No RBCS Family'
GROUP BY 1;
"""
)

`"No RBCS Family"` looks like a real, intentional value in our RBCS mapping, not a join failure.

Because:

- We have **RBCS_FamNumb = ‘000’** for every one of those rows (min and max both `000`), and the description is consistently `"No RBCS Family"`.
- If this were a join miss, we would typically see `RBCS_FamNumb` as `NULL` (or missing) after the join, not a stable sentinel like `000`.
- We also previously saw `pct_missing_rbcs = 0.0`, which strongly suggests the taxonomy file contains entries for these HCPCS codes and is assigning them to a “none/other/unclassified” bucket.

So it is probably a legitimate “catch-all” category: “this HCPCS code is in the taxonomy file but not assigned to a meaningful family.” But we have to confirm this finding.

Quick checks: How big is it relative to the dataset?

In [ ]:
con.sql(
    """
SELECT
  COUNT(*) AS n_all,
  SUM(CASE WHEN RBCS_FamNumb='000' THEN 1 ELSE 0 END) AS n_000,
  1.0 * SUM(CASE WHEN RBCS_FamNumb='000' THEN 1 ELSE 0 END) / COUNT(*) AS pct_000
FROM provider_service_features;
"""
)

Quick checks: Does it have very different cost/exposure and spend?

In [ ]:
con.sql(
    """
SELECT
  CASE WHEN RBCS_FamNumb='000' THEN '000_NoFamily' ELSE 'OtherFamilies' END AS grp,
  COUNT(*) AS n_rows,
  AVG(services) AS avg_services,
  quantile_cont(log_stdzd_amt_per_service, 0.50) AS med_log_stdzd_amt_per_service,
  quantile_cont(log_stdzd_amt_per_service, 0.95) AS p95_log_stdzd_amt_per_service
FROM eda_dataset
GROUP BY 1;
"""
)

Let's calculate the % of total standardized spend coming from RBCS_FamNumb = ‘000':

1. of the meta dataset `prov_svc_onco_core_prov_all_rbcs_nppes_tbl`

In [ ]:
con.sql(
    """
SELECT
  SUM(Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) AS total_stdzd_spend_all,
  SUM(CASE WHEN RBCS_FamNumb='000' THEN Tot_Srvcs * Avg_Mdcr_Stdzd_Amt ELSE 0 END) AS total_stdzd_spend_000,
  1.0 * SUM(CASE WHEN RBCS_FamNumb='000' THEN Tot_Srvcs * Avg_Mdcr_Stdzd_Amt ELSE 0 END)
      / NULLIF(SUM(Tot_Srvcs * Avg_Mdcr_Stdzd_Amt), 0) AS pct_stdzd_spend_000
FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl;
"""
)

2. of the EDA dataset `eda_dataset`:

In [ ]:
con.sql(
    """
SELECT
  SUM(stdzd_spend) AS total_stdzd_spend_all,
  SUM(CASE WHEN RBCS_FamNumb='000' THEN stdzd_spend ELSE 0 END) AS total_stdzd_spend_000,
  1.0 * SUM(CASE WHEN RBCS_FamNumb='000' THEN stdzd_spend ELSE 0 END)
      / NULLIF(SUM(stdzd_spend), 0) AS pct_stdzd_spend_000
FROM eda_dataset;
"""
)

This is exactly the kind of check that makes the story credible. Based on our results, I would treat RBCS_FamNumb = #### '000' as a real, meaningful segment and handle it explicitly.

**What these numbers are saying**

**Size**

- 000 is **7.9%** of rows (25,993 out of 329,571). That’s too big to ignore as a “quirk”, but small enough that it #### could distort things if it behaves differently.

**Exposure (services)**

- 000_NoFamily has **much higher average services** per row: ~12,234 vs ~2,836.
- That is a big structural difference. It suggests 000 rows are not “typical families”, they might be umbrella #### billing codes, very high volume codes, or catch-all codes that aggregate a lot of activity.

**Cost level (log standardized cost per service)**

- Median log cost is **lower** for 000 (3.38) vs others (4.29).
    - On the original scale, that’s roughly exp(3.38) ≈ 29 vs exp(4.29) ≈ 73 (this is approximate because we used log1p, but the conclusion holds).
- The **95th percentile is similar-ish** (6.14 vs 5.78). So the tail is not wildly different, but the center is.

So yes, I would call it meaningfully different. Even if the p95 is close, the median and the services volume difference are telling us this group is a different “kind” of row.





<br>

**Quick extra check before locking the decision**

The one thing I’d verify is whether 000 is concentrated in a specific place of service or provider type.

In [ ]:
con.sql(
    """
SELECT
  Place_Of_Srvc,
  provider_type,
  COUNT(*) AS n_rows,
  AVG(services) AS avg_services,
  quantile_cont(log_stdzd_amt_per_service, 0.50) AS med_log_stdzd
FROM eda_df
WHERE RBCS_FamNumb = '000'
GROUP BY 1,2
ORDER BY n_rows DESC;
"""
)

This table makes it very clear that **RBCS_FamNumb = '000'** is not just a harmless “misc” label. It behaves like multiple different sub-populations depending on place of service and provider type.

**What these result implies**

**1) The biggest driver is Place_Of_Srvc = O for HemOnc and MedOnc**

Look at these two rows:

- **O, Hematology-Oncology:** 10,741 rows, **avg_services ~ 22,765**, **median log cost ~ 1.16**
- **O, Medical Oncology:** 3,508 rows, **avg_services ~ 18,157**, **median log cost ~ 1.20**

That median log cost is extremely low relative to the rest of our dataset. These look like very high-volume, very low standardized cost-per-service rows, which is exactly the kind of pattern we get from “special” codes, aggregation-type codes, or something that is not comparable to a standard service family.

**2) The F rows look much more “normal”**

When Place_Of_Srvc = F, the median log costs are around ~4 to 6, and avg_services are tens, not tens of thousands. Example:

- **F, HemOnc:** avg_services ~ 79, med log ~ 4.10
- **F, RadOnc:** avg_services ~ 76, med log ~ 4.09
- **F, GynOnc:** avg_services ~ 39, med log ~ 6.24 (higher)

So 000 is not one thing. It’s at least two very different things depending on place of service.

**What to do with 000 now**

Two possible strategies:

**Option A (recommended): Keep 000, but model it explicitly with interactions**

Do all of this:

1. Add a flag
- is_no_rbcs_family = 1 if RBCS_FamNumb == '000' else 0
2. Add an interaction-type feature (either explicit or via model)
- For Ridge: we can explicitly create a feature like is_no_rbcs_family * is_place_O (or just rely on one-hot plus linear interaction if we build it).
- For XGBoost: we don’t need to manually create interactions. It will split on RBCS_FamNumb == '000' and Place_Of_Srvc naturally.
3. In EDA and reporting, always show metrics stratified by:
- is_no_rbcs_family and Place_Of_Srvc

This gives us the clean story: “000 codes behave differently, especially in outpatient settings. We keep them but let the model treat them differently.”

**Option B (cleanest benchmarking narrative): Separate-track modeling**

- **Main benchmark model:** train on RBCS_FamNumb != '000' only.
- **Secondary model:** train on RBCS_FamNumb == '000' only (and maybe even split by Place_Of_Srvc inside it).

This avoids mixing apples and oranges. It also prevents those huge-volume low-cost O rows from influencing the main model’s learned relationships.

**Which option fits our goal best?**

Since we want the headline metric to be **O/E standardized cost per service** and we want the benchmarking story to be clean, here’s the best alignment:

- If the report/dashboard is meant to be “benchmark real oncology service families,” then **Option B** is the cleanest.
- If the product goal is “give every provider-year-service row an expected cost,” then **Option A** is better.



<br>

**One more tiny QC that will make the decision obvious**

Let's check whether those crazy high avg_services rows also dominate total volume. If (O, HemOnc) and (O, MedOnc) are most of the services too, then separating 000 is usually the safest.

In [ ]:
con.sql(
    """
SELECT
  Place_Of_Srvc,
  provider_type,
  SUM(services) AS total_services,
  COUNT(*) AS n_rows,
  1.0 * SUM(services) / (SELECT SUM(services) FROM eda_df WHERE RBCS_FamNumb='000') AS pct_of_000_services
FROM eda_df
WHERE RBCS_FamNumb='000'
GROUP BY 1,2
ORDER BY total_services DESC;
"""
)

This confirms it very strongly.

**What this table says in plain English**

For the “No RBCS Family” bucket (RBCS_FamNumb = '000'):

- **Outpatient HemOnc (O, Hematology-Oncology)** accounts for **~76.9%** of *all* services in the 000 bucket.
- **Outpatient MedOnc (O, Medical Oncology)** accounts for **~20.0%**.
- Together, those two rows account for **~97%** of all 000 services.

Everything else is tiny by comparison.

So **RBCS_FamNumb='000'** is not just 7.9% of rows. It is a bucket whose *service volume* is overwhelmingly dominated by a very specific pattern (Outpatient + HemOnc/MedOnc). That means if we keep 000 in the main model without special handling, it can easily distort training, calibration, and the O/E story.

**Best recommendation**

Use **Option B (separate-track modeling)** as the default.

**Why Option B is the cleanest for our stated goal**

narrative usually assumes “we’re comparing like-with-like service families.” The 000 bucket is not behaving like a normal service family. It’s acting like a special aggregation or fallback code bucket with huge volume and very different cost behavior.

So:

1. **Main benchmark model (the one we show by default)**
- Train and score on **RBCS_FamNumb != '000'** only.
- This becomes our primary O/E story.
2. **Secondary model (optional add-on)**
- Train a separate model for **RBCS_FamNumb = '000'*, and honestly this could be restricted to just:
- (Place_Of_Srvc='O' AND provider_type IN ('Hematology-Oncology','Medical Oncology'))
- because that’s basically the whole 000 volume anyway.
- Or we can still include all provider types, but the key is keeping it separate from the main benchmark.

**What to do in the dataset**

Keep 000 rows in our data, but tag them and split the pipeline:

- model_track = 'main' if RBCS_FamNumb != '000'
- model_track = 'no_family' if RBCS_FamNumb = '000'

Then we build:

- model_dataset_main.parquet
- model_dataset_no_family.parquet (optional, can come later)



<br>

**One more quick QC that’s worth doing**

Check whether 000 is also concentrated in **specific HCPCS codes** (often a sign it’s a special code pattern):

If we see a small number of HCPCS codes dominating, that further supports “special bucket” behavior.

In [ ]:
con.sql(
    """
SELECT
  p.HCPCS_Cd,
  MAX(p.HCPCS_Desc) AS hcpcs_desc,
  COUNT(*) AS n_rows,
  SUM(p.Tot_Srvcs) AS total_services
FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl p
WHERE p.RBCS_FamNumb = '000'
GROUP BY 1
ORDER BY total_services DESC
LIMIT 20;
"""
)

It seems like most of the descriptions are injections. We should confirm this by running a regex to show what percentage of all **RBCS_FamNumb = '000'** are described as "injections" at any context of specific drug.

Fast check: “Injection” share by rows and by services

If pct_services_injection is very high (for example > 0.7), then the “000 bucket” is essentially “drug units/injections” in our data.

Slightly broader keyword net

Some injection-related codes might not literally contain the word “injection” (for example contrast materials). We can create a broader “drug or injectable” bucket:

This will tell us whether “injection” alone explains most of the services, or if it’s “injection + contrast + unclassified drug” etc.

In [ ]:
con.sql(
    """
WITH base AS (
  SELECT
    CASE
      WHEN lower(HCPCS_Desc) LIKE '%injection%' THEN 1
      ELSE 0
    END AS is_injection,
    Tot_Srvcs
  FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
  WHERE RBCS_FamNumb = '000'
)
SELECT
  COUNT(*) AS n_rows,
  SUM(is_injection) AS n_injection_rows,
  1.0 * SUM(is_injection) / COUNT(*) AS pct_rows_injection,

  SUM(Tot_Srvcs) AS total_services,
  SUM(CASE WHEN is_injection = 1 THEN Tot_Srvcs ELSE 0 END) AS injection_services,
  1.0 * SUM(CASE WHEN is_injection = 1 THEN Tot_Srvcs ELSE 0 END) / NULLIF(SUM(Tot_Srvcs), 0) AS pct_services_injection
FROM base;
"""
)

Bonus: Let's use `HCPCS_Drug_Ind` in our `prov_svc_onco_core_prov_all_rbcs_nppes_tbl`

Our big table appears to have `HCPCS_Drug_Ind`. If it’s reliable, it’s cleaner than regex:

**What this means for our modeling decision**

If we confirm that 000 is mostly drug-unit injection codes:

- It strengthens the case to **exclude RBCS_FamNumb='000' from the main service-family benchmark model** (our Option B main track).
- Then we can treat 000 as a **secondary “drug units” track** later, with its own modeling logic and careful interpretation.

In [ ]:
con.sql(
    """
WITH base AS (
  SELECT
    lower(HCPCS_Desc) AS d,
    Tot_Srvcs
  FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
  WHERE RBCS_FamNumb = '000'
),
tagged AS (
  SELECT
    CASE
      WHEN d LIKE '%injection%' THEN 'injection'
      WHEN d LIKE '%contrast%' THEN 'contrast'
      WHEN d LIKE '%unclassified drug%' OR d LIKE '%unclassified drugs%' THEN 'unclassified_drug'
      WHEN d LIKE '%chemotherap%' THEN 'chemo'
      WHEN d LIKE '%imaging%' OR d LIKE '%gadol%' OR d LIKE '%radiology%' THEN 'imaging_related'
      ELSE 'other'
    END AS bucket,
    Tot_Srvcs
  FROM base
)
SELECT
  bucket,
  COUNT(*) AS n_rows,
  SUM(Tot_Srvcs) AS total_services,
  1.0 * SUM(Tot_Srvcs) / (SELECT SUM(Tot_Srvcs) FROM tagged) AS pct_services
FROM tagged
GROUP BY 1
ORDER BY total_services DESC;
"""
)

In [ ]:
con.sql(
    """
SELECT
  HCPCS_Drug_Ind,
  COUNT(*) AS n_rows,
  SUM(Tot_Srvcs) AS total_services,
  1.0 * SUM(Tot_Srvcs) / (SELECT SUM(Tot_Srvcs)
                          FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
                          WHERE RBCS_FamNumb='000') AS pct_services
FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
WHERE RBCS_FamNumb='000'
GROUP BY 1
ORDER BY total_services DESC;
"""
)

It seems that most of the RBCS_FamNumb='000'  are actually related to injections.

These three checks line up perfectly, and they give us a very defensible conclusion.
What our results mean

1) “Injection” dominates by services (the important lens)

<br>

Only 53.4% of rows contain the word “injection”.
But those rows account for 91.3% of total services in the 000 bucket.
So the 000 bucket is not “half injections” in the sense that matters for modeling. It’s overwhelmingly injection volume.

<br>

2) Our broader keyword tagging confirms the same story

Injection: 91.3% of services
Contrast: 6.8% of services
Everything else combined is small.
This is very consistent with “drug units and related items” rather than standard “service families.”

<br>

3) The drug indicator seals it

<br>

HCPCS_Drug_Ind = 'Y' is 98.5% of services in 000.
That is huge. It means RBCS_FamNumb='000' is basically a drug-related bucket in our extracted oncology universe.
What we should do (clear recommendation)
Use the split-track approach:
Main benchmarking model (our portfolio headline)
Train and score ONLY on rows where:
RBCS_FamNumb IS NOT NULL
RBCS_FamNumb <> '000'
This is our clean “apples-to-apples service-family benchmarking engine.”
Secondary “drug unit” track (optional add-on)

<br>

Keep RBCS_FamNumb='000' rows separate.
If we include it later, frame it honestly as “drug unit and related codes (injections, contrast).”
It can be its own model or a descriptive module. It does not need to be part of the core benchmark story.
This aligns with our goal: make the primary narrative clean and interview-safe for Garner/Oscar.
How to describe this decision in our write-up

<br>

We can say something like:
“RBCS family ‘000’ represents non-standard family assignments in RBCS. In our oncology subset, it is overwhelmingly drug-related volume: 98.5% of services have HCPCS_Drug_Ind='Y', and 91.3% of services are codes whose description includes ‘injection’. Because drug unit billing behaves differently from service-family costs, we exclude RBCS ‘000’ from the main service-family benchmark model and treat it as a separate track.”
That sounds thoughtful, not like data cleaning hand-waving.

Let's now calculate within `RBCS_FamNumb='000'` what is the % of standardized spend from injection vs other

A) Text-based (“injection” in description), using the HCPCS table

In [ ]:
con.sql(
    """
WITH base AS (
  SELECT
    CASE WHEN lower(HCPCS_Desc) LIKE '%injection%' THEN 'injection' ELSE 'other' END AS bucket,
    Tot_Srvcs * Avg_Mdcr_Stdzd_Amt AS stdzd_spend
  FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
  WHERE RBCS_FamNumb='000'
)
SELECT
  bucket,
  COUNT(*) AS n_rows,
  SUM(stdzd_spend) AS total_stdzd_spend,
  1.0 * SUM(stdzd_spend) / NULLIF((SELECT SUM(stdzd_spend) FROM base), 0) AS pct_stdzd_spend
FROM base
GROUP BY 1
ORDER BY total_stdzd_spend DESC;
"""
)



B) Drug-indicator-based (often cleaner than text):

This uses `HCPCS_Drug_Ind` variable instead of relying on character matching.

In [ ]:
con.sql(
    """
WITH base AS (
  SELECT
    CASE WHEN HCPCS_Drug_Ind='Y' THEN 'drug_ind_Y' ELSE 'drug_ind_N' END AS bucket,
    Tot_Srvcs * Avg_Mdcr_Stdzd_Amt AS stdzd_spend
  FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
  WHERE RBCS_FamNumb='000'
)
SELECT
  bucket,
  COUNT(*) AS n_rows,
  SUM(stdzd_spend) AS total_stdzd_spend,
  1.0 * SUM(stdzd_spend) / NULLIF((SELECT SUM(stdzd_spend) FROM base), 0) AS pct_stdzd_spend
FROM base
GROUP BY 1
ORDER BY total_stdzd_spend DESC;
"""
)



**The interesting part: why “injection dominates services” but not “spend”**

Earlier, we found within 000:

- ~**98% of services** are “injection” (text match), and ~**98.6% of services** have `HCPCS_Drug_Ind='Y'`.

But now we see **only ~42% to 45% of spend** is injection or drug-indicated.

That usually means: **the injection/drug rows have massive unit counts (Tot_Srvcs) but low standardized amount per unit**, while the non-injection or non-drug rows have fewer units but higher standardized amounts per unit.

To make that relationship explicit, let's run this:

That will quantify the “lots of units, cheap per unit” vs “few units, expensive per unit” pattern.

In [ ]:
con.sql(
    """
WITH base AS (
  SELECT
    CASE WHEN lower(HCPCS_Desc) LIKE '%injection%' THEN 'injection' ELSE 'other' END AS bucket,
    Tot_Srvcs AS services,
    (Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) AS stdzd_spend
  FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
  WHERE RBCS_FamNumb='000'
)
SELECT
  bucket,
  SUM(services) AS total_services,
  SUM(stdzd_spend) AS total_stdzd_spend,
  SUM(stdzd_spend) / NULLIF(SUM(services), 0) AS implied_stdzd_amt_per_service,
  1.0 * SUM(services) / NULLIF((SELECT SUM(services) FROM base),0) AS pct_services,
  1.0 * SUM(stdzd_spend) / NULLIF((SELECT SUM(stdzd_spend) FROM base),0) AS pct_spend
FROM base
GROUP BY 1
ORDER BY total_stdzd_spend DESC;
"""
)

What each column is telling us

Bucket totals
- Injection
    - total_services: 240,415,048.9 (98.13% of services)
    - total_stdzd_spend: $287,613,028.58 (45.36% of spend)
    - implied_stdzd_amt_per_service: $1.20 per service
- Other
    - total_services: 4,573,314.8 (1.87% of services)
    - total_stdzd_spend: $346,436,797.62 (54.64% of spend)
    - implied_stdzd_amt_per_service: $75.75 per service

The main takeaway

Within RBCS_FamNumb='000':
- Almost all of the service volume is “injection” (98.13%).
- But most of the standardized spend is “other” (54.64%).
- The reason is the per-service cost difference:
    - “Injection” averages ~$1.20 per unit
    - “Other” averages ~$75.75 per unit
    - That’s about a 63x difference (75.75 / 1.20 ≈ 63.3).




Those huge injection service counts are often because many drug HCPCS codes are billed in tiny units (mg, mcg, 1 mg units, etc.). To show that explicitly, let's pull the top injection HCPCS in 000 by services and look at their descriptions:

In [ ]:
con.sql(
    """
SELECT
  HCPCS_Cd,
  MAX(HCPCS_Desc) AS hcpcs_desc,
  SUM(Tot_Srvcs) AS total_services,
  SUM(Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) AS total_stdzd_spend,
  SUM(Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) / NULLIF(SUM(Tot_Srvcs),0) AS implied_amt_per_service
FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
WHERE RBCS_FamNumb='000'
  AND lower(HCPCS_Desc) LIKE '%injection%'
GROUP BY 1
ORDER BY total_services DESC
LIMIT 20;
"""
)

Let's quantify how much of injection volume comes from “per 1 mg / microgram / mcg / 0.1 ml” unit-style descriptions:

In [ ]:
con.sql(
    """
WITH inj AS (
  SELECT
    lower(HCPCS_Desc) AS d,
    Tot_Srvcs AS services
  FROM prov_svc_onco_core_prov_all_rbcs_nppes_tbl
  WHERE RBCS_FamNumb='000'
    AND lower(HCPCS_Desc) LIKE '%injection%'
)
SELECT
  CASE
    WHEN d LIKE '% 1 mg%' OR d LIKE '%per 1 mg%' THEN 'per_1mg'
    WHEN d LIKE '%microgram%' OR d LIKE '% mcg%' THEN 'microgram_or_mcg'
    WHEN d LIKE '% 0.1 ml%' THEN 'per_0.1ml'
    ELSE 'other_injection_units'
  END AS unit_bucket,
  SUM(services) AS total_services,
  1.0 * SUM(services) / (SELECT SUM(services) FROM inj) AS pct_injection_services
FROM inj
GROUP BY 1
ORDER BY total_services DESC;
"""
)

## EDA 2. Outcome distributions and log justification

**Plots**

1. Histogram: `stdzd_amt_per_service` (raw)
2. Histogram: `log_stdzd_amt_per_service`
3. Box plot: `stdzd_amt_per_service` by `Place_Of_Srvc` (overall)
4. Percentiles table for standardized, allowed, payment, submitted:
    - p01, p50, p90, p95, p99, max

**Questions answered**

- How skewed are costs?
- Does log transform stabilize the distribution?
- Are there extreme outliers that will dominate training if we do not log?

In [ ]:
plt.hist(eda_df["stdzd_amt_per_service"])
plt.title("Histogram of stdzd_amt_per_service variable (original scale)")
plt.show()

In [ ]:
plt.hist(is_bottom_99pct_eda_df["stdzd_amt_per_service"])
plt.title("Histogram of stdzd_amt_per_service variable (original scale) (bottom 99%)")
plt.show()

In [ ]:
plt.hist(np.log1p(eda_df["stdzd_amt_per_service"]))
plt.title("Histogram of stdzd_amt_per_service variable (np.log1p-transformed scale)")
plt.show()

In [ ]:
g = sns.boxplot(data=eda_df,
                x = "Place_Of_Srvc",
                y="log_stdzd_amt_per_service")
g.set_title("log_stdzd_amt_per_service by Place_Of_Srvc")
plt.show()

In [ ]:
cols = ["stdzd_amt_per_service", "allowed_amt_per_service", "payment_amt_per_service"]

percentiles = eda_df[cols].quantile([0.01, 0.5, 0.9, 0.95, 0.99])

max_vals = eda_df[cols].max().to_frame().T
max_vals.index = [1.0]

result = pd.concat([percentiles, max_vals])
print(result)



In [ ]:
eda_df.columns

Let's also calculate share of total spend contributed by top 1% rows” and “share of services contributed by top 1%”.

In [ ]:
(eda_df
 .groupby("is_top_1pct_stdzd_amt_per_service")
 .agg(
     total_spend=("stdzd_spend", "sum"),
     total_services=("services", "sum"),
 )
 .assign(
     pct_total_spend=lambda x: x["total_spend"] / x["total_spend"].sum(),
     pct_total_services=lambda x: x["total_services"] / x["total_services"].sum(),
 ))

## EDA 3. Exposure, reliability, and threshold selection

**Plots**

1. Histogram: services
2. CDF plot: cumulative share of rows by services
3. Scatter: `services` vs `stdzd_amt_per_service` (use log x-axis)
4. Optional: binned variance plot
    - bin services into ranges and show cost variance per bin

**Questions answered**

- How much of the dataset is low-volume and noisy?
- Is services >= 11 reasonable for training?
- What threshold should we use later for “flagging” outliers (like 50+)?

Histogram of services: The distribution of exposure (how many services went into each row’s per-service cost).


In [ ]:
plt.hist(eda_df["services"])
plt.title("Histogram of services variable")
plt.show()

The histogram is xtremely right-skewed. Most rows have relatively low services, and a small number of rows have massive service counts (up to ~500k). That is normal in claims-like utilization data.

Low-service rows have noisier “per service” averages. High-service rows have very stable averages.

In [ ]:
sns.ecdfplot(data = eda_df, x="services")
plt.title("CDF of Services: Cumulative Share of Rows")
plt.show()

In [ ]:
eda_df["log_services"] = np.log1p(eda_df["services"])

g = sns.scatterplot(data=eda_df, x = "log_services", y="log_stdzd_amt_per_service", alpha=0.1, s=10)
g.set_title("log_stdzd_amt_per_service vs log_services")
plt.show()

In [ ]:
# 1. Create a binned version of 'services' (e.g., 5 equal-width bins)
eda_df['services_bins'] = pd.cut(eda_df['services'], bins=5)

# 2. Plot variance per bin
plt.figure(figsize=(10, 6))
sns.barplot(data=eda_df, x='services_bins', y='log_stdzd_amt_per_service', estimator=np.var)

plt.title('Cost Variance per Services Bin')
plt.ylabel('Variance of log_stdzd_amt_per_service')
plt.xlabel('Services Bins')
plt.show()

In [ ]:
# Define custom edges: 0 to 11, 11 to 25, 25 to 50, 50 to 100, and 100+
custom_bins = [0, 11, 25, 50, 100, np.inf]
custom_labels = ['0-11', '12-25', '26-50', '51-100', '100+']

# Create the binned column
eda_df['services_custom'] = pd.cut(
    eda_df['services'], 
    bins=custom_bins, 
    labels=custom_labels, 
    include_lowest=True
)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    data=eda_df, 
    x='services_custom', 
    y='log_stdzd_amt_per_service', 
    estimator=np.var
)

plt.title('Cost Variance by Custom Service Thresholds')
plt.ylabel('Variance (log_stdzd_amt_per_service)')
plt.xlabel('Services Bins')
plt.show()


**Adding counts per bin**

We need to know if 100+ has a small number of rows (unstable estimate) or tons (real pattern).

In [ ]:
eda_df.groupby("services_custom").agg(
    n_rows=("log_stdzd_amt_per_service","size"),
    var_y=("log_stdzd_amt_per_service", np.var),
    mean_y=("log_stdzd_amt_per_service","mean"),
    p95=("log_stdzd_amt_per_service", lambda s: np.quantile(s, 0.95))
)

**Split 100+ into sub-bins**

Keep our early bins, but break 100+ into a few more. For example:

- 101–250
- 251–1,000
- 1,001–10,000
- 10,000+

If the variance spike is really coming from, say, the “10k+” group, that is an important story.

In [ ]:
bins2 = [0, 11, 25, 50, 100, 250, 1000, 10000, np.inf]
labels2 = ["0-11","12-25","26-50","51-100","101-250","251-1k","1k-10k","10k+"]

eda_df["services_custom2"] = pd.cut(eda_df["services"], bins=bins2, labels=labels2, include_lowest=True)

sns.barplot(data=eda_df, x="services_custom2", y="log_stdzd_amt_per_service", estimator=np.var)
plt.xticks(rotation=30, ha="right")
plt.show()

**1) The “100+” bin is not “more reliable”. It’s a different regime.**

Look at these side by side:

- **51–100:** var ≈ **0.679**, IQR ≈ **0.780**, mean ≈ **4.47**, p95 ≈ **5.80**
- **100+:** var ≈ **1.874**, IQR ≈ **1.571**, mean ≈ **3.51**, p95 ≈ **5.04**

So 100+ is:

- **much more spread out** (variance and IQR both jump a lot),
- with a **lower mean** and **lower p95** than the smaller bins.

That means the huge variance is not just a few outliers. It’s telling us that 100+ rows are a heterogeneous mix (different families, different billing patterns, different settings), which is exactly what the sub-bin plot is showing.

**2) The split confirms the variance spike is concentrated in a middle-high range**

From the plot:

- 0–11 through 51–100 are all in the same ballpark (stable-ish).
- **101–250 and 251–1k** start drifting up a bit.
- **1k–10k** is the giant spike (the main problem).
- **10k+** drops again (still higher than low bins, but much lower than 1k–10k).

That is a classic “mixture” story: the 1k–10k segment likely contains specific high-volume drug administration patterns that have very different per-service cost behavior.

**Let's also use IQR as a robustness check**

Variance can be misleading. IQR tells us the “middle spread.”

In [ ]:
def iqr(s):
    return np.quantile(s, 0.75) - np.quantile(s, 0.25)

eda_df.groupby("services_custom").agg(
    n_rows=("log_stdzd_amt_per_service","size"),
    iqr_y=("log_stdzd_amt_per_service", iqr),
    var_y=("log_stdzd_amt_per_service", np.var)
)

In [ ]:
eda_df.groupby("services_custom2").agg(
    n_rows=("log_stdzd_amt_per_service","size"),
    iqr_y=("log_stdzd_amt_per_service", lambda s: np.quantile(s,0.75)-np.quantile(s,0.25)),
    var_y=("log_stdzd_amt_per_service", np.var)
    
)

**The key takeaway**

Our dataset is **row-heavy in the low and mid volumes**, but **service-heavy in the ultra-high volumes**.

- **10k+ is only 7.27% of rows** (23,947 rows)
- But it contains **84.17% of all services** (992M services)

So if we care about “what drives total oncology service volume”, the 10k+ group dominates. If we we about “typical provider-service records”, the smaller bins dominate.

This is exactly why we need a deliberate weighting strategy and why we should report results stratified by volume bin.

In [ ]:
tmp = (
    eda_df.groupby("services_custom2")
    .agg(
        n_rows=("services","size"),
        total_services=("services","sum")
    )
)
tmp["pct_rows"] = tmp["n_rows"] / tmp["n_rows"].sum()
tmp["pct_services"] = tmp["total_services"] / tmp["total_services"].sum()
tmp.sort_index()

## EDA 4. Service mix and spend concentration (core storytelling)

**Plots**

1. Top 15 families by total services:
    - bar: `SUM(services)` by `rbcs_family_desc`
2. Top 15 families by total standardized spend:
    - bar: `SUM(stdzd_spend)` by `rbcs_family_desc`
3. Pareto chart:
    - cumulative percent of spend captured by top families

**Questions answered**

- Which service families drive volume?
- Which service families drive spend?
- Where is the biggest opportunity for steerage?

**1A. Top 15 families by total services (the entire `eda_df` dataset)**

In [ ]:
sum_srvc_by_rbcs_family_desc = (eda_df.groupby("rbcs_family_desc", as_index=False)
          .agg(services_sum=("services", "sum"))
          .sort_values("services_sum", ascending=False))

sum_srvc_by_rbcs_family_desc[:15]

This table is already telling a clear story, and the “No RBCS Family” row is not a red flag anymore given what we already proved earlier.

**How to interpret the table**

Each row is an **RBCS family bucket** (a service category), and services_sum is the **total number of services** in our EDA dataset for that family.

So this is basically answering:

**“Which oncology service families account for the most delivered volume?”**

**What jumps out immediately**

1) The top of the list is dominated by “drug administration and supportive meds”

- Erythropoiesis stimulating agents (largest)
- Chemotherapeutic agents
- Injection colony stimulating factors
- Injection administration
- IV infusion, hydration
- Immune globulin injections

That pattern is very consistent with outpatient oncology operations (lots of repeated drug-related units).

2) “No RBCS Family” being #2 is expected, and it has a specific meaning in our data

From our earlier QC:

- rbcs_family_desc = 'No RBCS Family' corresponds to **RBCS_FamNumb = '000'**
- And **~84% of services inside that group are injections**, and **~98% of its services are HCPCS_Drug_Ind = ‘Y’**

So we should interpret **“No RBCS Family”** as:

> “A large set of mostly drug-related injection HCPCS codes that are not assigned to a standard RBCS family in the mapping.”

It is not “missing data” in the usual sense. It is a real bucket that our RBCS mapping defines as “unclassified or not mapped to a standard family”.

3) Evaluation & Management (E&M) is high volume, not surprising

- Office E&M new
- Office E&M established

These are common visit codes and will produce lots of service counts even if cost per service is not huge.

4) Pathology and imaging show up as high volume too

- Surgical pathology examination
- Contrast agent
- Plus radiation-related families

That is plausible because oncology care triggers a lot of labs, imaging, and radiation planning.

**1B. Top 15 families by total services (the top 1% for standardized cost per service part of the `eda_df` dataset)**

In [ ]:
sum_srvc_by_rbcs_family_desc = (is_top_1pct_eda_df.groupby("rbcs_family_desc", as_index=False)
          .agg(services_sum=("services", "sum"))
          .sort_values("services_sum", ascending=False))

sum_srvc_by_rbcs_family_desc[:15]

**1C. Top 15 families by total services (the bottom 99% for standardized cost per service part of the `eda_df` dataset)**

In [ ]:
sum_srvc_by_rbcs_family_desc = (is_bottom_99pct_eda_df.groupby("rbcs_family_desc", as_index=False)
          .agg(services_sum=("services", "sum"))
          .sort_values("services_sum", ascending=False))

sum_srvc_by_rbcs_family_desc[:15]

**2A. Top 15 families by total standardized spend (the entire `eda_df` dataset)**

In [ ]:
sum_stdzd_spend_by_rbcs_family_desc = (eda_df
                                       .groupby("rbcs_family_desc", as_index=False)
                                       .agg(sum_stdzd_spend = ("stdzd_spend","sum"))
                                       .assign(sum_stdzd_spend_pct = lambda x: x["sum_stdzd_spend"]/x["sum_stdzd_spend"].sum())
                                       .sort_values("sum_stdzd_spend", ascending=False))

sum_stdzd_spend_by_rbcs_family_desc[:15]

In [ ]:
# 1) Spend by family
fam = (eda_df.groupby("rbcs_family_desc", as_index=False)
       .agg(sum_stdzd_spend=("stdzd_spend", "sum"))
       .sort_values("sum_stdzd_spend", ascending=False))

# 2) Cumulative percent of spend
fam["cum_spend"] = fam["sum_stdzd_spend"].cumsum()
total_spend = fam["sum_stdzd_spend"].sum()
fam["cum_pct_spend"] = fam["cum_spend"] / total_spend

# 3) Pick top N to display
N = 25
top = fam.head(N).copy()

# 4) Plot: bars = spend, line = cumulative percent
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(top["rbcs_family_desc"], top["sum_stdzd_spend"])
ax1.set_ylabel("Total standardized spend (sum)")
ax1.set_title(f"Pareto of standardized spend by RBCS family (Top {N})")
ax1.tick_params(axis="x", rotation=75)

ax2 = ax1.twinx()
ax2.plot(top["rbcs_family_desc"], top["cum_pct_spend"] * 100, marker="o")
ax2.set_ylabel("Cumulative % of total spend")

# Optional reference lines (80/20 rule vibe)
ax2.axhline(80, linestyle="--")
ax2.axhline(90, linestyle="--")

plt.tight_layout()
plt.show()

# Optional: show the key numbers as a table
top[["rbcs_family_desc", "sum_stdzd_spend", "cum_pct_spend"]].head(15)

In [ ]:
# 1) Spend by family
fam = (eda_df.groupby("rbcs_family_desc", as_index=False)
       .agg(sum_stdzd_spend=("stdzd_spend", "sum"))
       .sort_values("sum_stdzd_spend", ascending=False))

# 2) Cumulative percent of spend
fam["cum_spend"] = fam["sum_stdzd_spend"].cumsum()
total_spend = fam["sum_stdzd_spend"].sum()
fam["cum_pct_spend"] = fam["cum_spend"] / total_spend

# fam must already be sorted desc by sum_stdzd_spend and have cum_pct_spend computed
N = 25
top = fam.head(N).copy()

# Reverse so biggest is at top in barh
top = top.iloc[::-1].reset_index(drop=True)

y = np.arange(len(top))

fig, ax1 = plt.subplots(figsize=(12, 8))

# Bars (spend)
ax1.barh(y, top["sum_stdzd_spend"])
ax1.set_yticks(y)
ax1.set_yticklabels(top["rbcs_family_desc"], fontsize=9)
ax1.set_xlabel("Total standardized spend (sum)")
ax1.set_title(f"Pareto of standardized spend by RBCS family (Top {N})")

# Line (cumulative percent) on a top x-axis or secondary x-axis
ax2 = ax1.twiny()
ax2.plot(top["cum_pct_spend"] * 100, y, marker="o", c = "red")
ax2.set_xlabel("Cumulative % of total spend")

# Reference lines
ax2.axvline(80, linestyle="--")
ax2.axvline(90, linestyle="--")

plt.tight_layout()
plt.show()

In [ ]:
# 1) Spend by family
eda_mapped = eda_df[eda_df["RBCS_FamNumb"] != "000"].copy()

fam = (eda_mapped.groupby("rbcs_family_desc", as_index=False)
       .agg(sum_stdzd_spend=("stdzd_spend", "sum"))
       .sort_values("sum_stdzd_spend", ascending=False))

# 2) Cumulative percent of spend
fam["cum_spend"] = fam["sum_stdzd_spend"].cumsum()
total_spend = fam["sum_stdzd_spend"].sum()
fam["cum_pct_spend"] = fam["cum_spend"] / total_spend

# 3) Pick top N to display
N = 25
top = fam.head(N).copy()

# 4) Plot: bars = spend, line = cumulative percent
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(top["rbcs_family_desc"], top["sum_stdzd_spend"])
ax1.set_ylabel("Total standardized spend (sum)")
ax1.set_title(f"Pareto of standardized spend by RBCS family (Top {N}) ('RBCS_FamNumb =! 000')")
ax1.tick_params(axis="x", rotation=75)

ax2 = ax1.twinx()
ax2.plot(top["rbcs_family_desc"], top["cum_pct_spend"] * 100, marker="o")
ax2.set_ylabel("Cumulative % of total spend")

# Optional reference lines (80/20 rule vibe)
ax2.axhline(80, linestyle="--")
ax2.axhline(90, linestyle="--")

plt.tight_layout()
plt.show()

# Optional: show the key numbers as a table
top[["rbcs_family_desc", "sum_stdzd_spend", "cum_pct_spend"]].head(15)

In [ ]:
# 1) Spend by family
eda_mapped = eda_df[eda_df["RBCS_FamNumb"] != "000"].copy()

fam = (eda_mapped.groupby("rbcs_family_desc", as_index=False)
       .agg(sum_stdzd_spend=("stdzd_spend", "sum"))
       .sort_values("sum_stdzd_spend", ascending=False))

# 2) Cumulative percent of spend
fam["cum_spend"] = fam["sum_stdzd_spend"].cumsum()
total_spend = fam["sum_stdzd_spend"].sum()
fam["cum_pct_spend"] = fam["cum_spend"] / total_spend

# fam must already be sorted desc by sum_stdzd_spend and have cum_pct_spend computed
N = 25
top = fam.head(N).copy()

# Reverse so biggest is at top in barh
top = top.iloc[::-1].reset_index(drop=True)

y = np.arange(len(top))

fig, ax1 = plt.subplots(figsize=(12, 8))

# Bars (spend)
ax1.barh(y, top["sum_stdzd_spend"])
ax1.set_yticks(y)
ax1.set_yticklabels(top["rbcs_family_desc"], fontsize=9)
ax1.set_xlabel("Total standardized spend (sum)")
ax1.set_title(f"Pareto of standardized spend by RBCS family (Top {N}) ('RBCS_FamNumb != 000')")

# Line (cumulative percent) on a top x-axis or secondary x-axis
ax2 = ax1.twiny()
ax2.plot(top["cum_pct_spend"] * 100, y, marker="o", c = "red")
ax2.set_xlabel("Cumulative % of total spend")

# Reference lines
ax2.axvline(80, linestyle="--")
ax2.axvline(90, linestyle="--")

plt.tight_layout()
plt.show()

We implemented the Pareto correctly, and the outputs are telling a very clear story.

**How to interpret the “Top families by total services” table**

That table is basically answering: **“What kinds of things are being done the most often (by count of services)?”**

- **Services_sum is volume**, not dollars.
- So it is totally possible for something to be huge in services but not huge in spend (cheap but frequent), and vice versa (expensive but not frequent).

**Why “No RBCS Family” is so high in services**

Given what we already proved earlier, **RBCS_FamNumb = ‘000’ is mostly injection-related and drug-indicated** (about ~91% of services have “injection” in the description, and ~98% of services are HCPCS_Drug_Ind = Y). That usually implies:

- These are **drug J-codes and related admin-type codes** that our RBCS mapping is not assigning to a regular family, so they fall into the **“000 / No family” bucket**.
- It is likely a **real “unmapped/other” bucket**, not random noise, but it is also **not clinically interpretable as a real service family** like “Chemo agent” or “Radiation planning”.

So interpret it as: **“A big chunk of volume is in drug injection codes that are not mapped into a specific RBCS family.”**

That is not automatically “bad”. It just means we should treat it intentionally in EDA and modeling (more on that below).

**How to interpret the “Top families by total standardized spend” table**

This table answers: **“Where is the money?”**

- Here, **Chemotherapeutic Agent is #1**, which makes sense because chemo drugs are expensive even if not always the #1 by pure service count.
- **Office E&M** categories being near the top is also common because they combine meaningful volume with non-trivial cost.
- **Radiation families** show up strongly because they are high-cost and systematic.

**Where “No RBCS Family” fits here**

It is **still large** (around 0.93B in our full table, top 6), but it becomes **less dominant than the big clinical families**.

That matches our earlier diagnosis:

- “000” is **massive volume**, but its **median log cost is lower than other families**.
- Still, because volume is huge, it can become **material spend**.

**How to interpret the Pareto tables and plots (the key story)**

A Pareto answers: **“How concentrated is spend in a small number of families?”**

From our table **including** “No RBCS Family”:

- Top 1 family (Chemo agent) already gets us to **~23.5%** of spend.
- Top 5 gets us to **~63.4%**.
- Top 8 gets us to **~79.8%** (we’re basically at the 80% line).
- Top 12 gets us to **~90.1%**.

That is a very strong concentration story: **a small set of service families explains most total standardized spend**.

When we **exclude** RBCS_FamNumb = ‘000’, the curve gets even “cleaner” conceptually:

- Top 5 gets us to **~67.4%**.
- Top 7 gets us to **~78.5%**.
- Top 11 gets us to **~89.5%**.

So “000” is not driving the whole Pareto story. The story holds either way.

**What I would do with “No RBCS Family” going forward**

We have two defensible paths. Pick one and be consistent.

**Option 1 (recommended for clean storytelling)**

**Exclude FamNumb = ‘000’ from the “service mix and spend concentration” plots**, and explicitly label it as “unmapped drug injection codes” elsewhere.

Why:

- Those plots are meant to tell a clinical/operational story about families.
- “No RBCS Family” is not interpretable in the same way.

We can still keep it in the dataset for modeling if we want, but keep the family-level “mix” charts clean.

**Option 2 (keep it, but treat it as its own explicit bucket)**

Keep it in plots, but **rename it in charts** to something like:

- “Unmapped (000) drug injection codes”
    
    so readers do not think it is a meaningful service family.
    

**Quick next EDA steps that fit perfectly after our Pareto**

1. **Make a second Pareto by Place_Of_Srvc** (F vs O).
    
    Question it answers: “Is spend concentration different in facility vs office settings?”
    
2. **Make the Pareto within provider_type** (HemOnc vs RadOnc, etc.).
    
    Question it answers: “Are different specialties driven by different spend families?”
    
3. **Add one table: top 10 families by spend, with median and p95 of log_stdzd_amt_per_service**
    
    Question it answers: “Which families are expensive because of unit cost vs expensive because of volume?”
    

If we want, paste our `eda_df` column list (or the EDA extract SQL we used), and I’ll give us the exact code blocks for these three so they drop into the notebook cleanly.

**2B. Top 15 families by total standardized spend (the top 1% for standardized cost per service part of the `eda_df` dataset)** 

In [ ]:
sum_stdzd_spend_by_rbcs_family_desc = (is_top_1pct_eda_df
                                       .groupby("rbcs_family_desc", as_index=False)
                                       .agg(sum_stdzd_spend = ("stdzd_spend","sum"))
                                       .assign(sum_stdzd_spend_pct = lambda x: x["sum_stdzd_spend"]/x["sum_stdzd_spend"].sum())
                                       .sort_values("sum_stdzd_spend", ascending=False))

sum_stdzd_spend_by_rbcs_family_desc[:15]

**1C. Top 15 families by total standardized spend (the bottom 99% for standardized cost per service part of the `eda_df` dataset)**

In [ ]:
sum_stdzd_spend_by_rbcs_family_desc = (is_bottom_99pct_eda_df
                                       .groupby("rbcs_family_desc", as_index=False)
                                       .agg(sum_stdzd_spend = ("stdzd_spend","sum"))
                                       .assign(sum_stdzd_spend_pct = lambda x: x["sum_stdzd_spend"]/x["sum_stdzd_spend"].sum())
                                       .sort_values("sum_stdzd_spend", ascending=False))

sum_stdzd_spend_by_rbcs_family_desc[:15]

**One small improvement to make this airtight:**

Let's add one more table that quantifies “tail enrichment” per family:

For each rbcs_family_desc, compute:

- share of spend in tail
- share of spend overall
- enrichment ratio = tail_share / overall_share

In [ ]:
# 1) totals for normalization
total_spend_all  = eda_df["stdzd_spend"].sum()
total_spend_tail = eda_df.loc[eda_df["is_top_1pct_stdzd_amt_per_service"], "stdzd_spend"].sum()

# 2) family-level spend (all vs tail)
fam_all = (eda_df.groupby("rbcs_family_desc", as_index=False)
           .agg(spend_all=("stdzd_spend", "sum")))

fam_tail = (eda_df.loc[eda_df["is_top_1pct_stdzd_amt_per_service"]]
            .groupby("rbcs_family_desc", as_index=False)
            .agg(spend_tail=("stdzd_spend", "sum")))

# 3) combine + compute shares and enrichment
tail_enrichment = (fam_all.merge(fam_tail, on="rbcs_family_desc", how="left")
                   .assign(
                       spend_tail=lambda d: d["spend_tail"].fillna(0.0),
                       overall_share=lambda d: d["spend_all"] / total_spend_all,
                       tail_share=lambda d: np.where(
                           total_spend_tail > 0, d["spend_tail"] / total_spend_tail, np.nan
                       ),
                       enrichment_ratio=lambda d: d["tail_share"] / d["overall_share"],
                       pct_spend_in_tail=lambda d: np.where(
                           d["spend_all"] > 0, d["spend_tail"] / d["spend_all"], np.nan
                       ),
                   )
                   .sort_values("enrichment_ratio", ascending=False))

# look at the most tail-enriched families
tail_enrichment.head(15)

The table above has, for each `rbcs_family_desc`:

- `overall_share` = “Out of all standardized spend in the full dataset, what fraction belongs to this family?”

- `tail_share` = “Out of standardized spend within the tail only (top 1% rows by `stdzd_amt_per_service`), what fraction belongs to this family?”

- `enrichment_ratio` = `tail_share / overall_share`
    - ">1" means the family is overrepresented in the tail.
    - "<1" means the family is underrepresented in the tail.

- `pct_spend_in_tail` = “Of this family’s total spend, what percent is in the tail?”

**A. The tail is not “spread around”. It is heavily concentrated in `PET- Oncology`.**

Look at `PET- Oncology`:

- `tail_share` = 0.580777 (about 58% of all tail spend)
- `overall_share` = 0.02233379 (about 2.23% of all spend)
- `enrichment_ratio` = 26.0 (about 26x overrepresented in the tail)
- `pct_spend_in_tail` = 0.450119 (about 45% of PET spend is in the tail)

*This tells a very clean story: the high-cost-per-service tail is largely a PET story.*

**B. Some families are “almost entirely tail”, but they are tiny overall.**

Examples:

- `Vascular Embolization` has `pct_spend_in_tail` ~ 0.99, enrichment ~57x
- `Arthroplasty Knee` and `Arthroplasty Hip` have `pct_spend_in_tail` ~ 0.96 and 0.94, enrichment ~55x

*This means: these families have very small total spend (spend_all is tiny compared to PET or IMRT), but when they show up, they tend to show up as extreme cost-per-service rows.*

These are “spiky” families. Great for interpretability, not necessarily major drivers of overall spend.

**C. The huge spend families are not tail-driven.**

Look at big baseline spend families:

- Conventional Radiation Treatment

    - `overall_share` ~ 9.56%
    - `tail_share` ~ 3.48%
    - `enrichment_ratio` ~ 0.36 (underrepresented in tail)
    - `pct_spend_in_tail` ~ 0.63%

- IMRT

- `overall_share` ~ 8.32%
- `tail_share` ~ 4.84%
- `enrichment_ratio` ~ 0.58
- `pct_spend_in_tail` ~ 1.0%

So our earlier EDA story that “these are major spend families” remains true, and it is not being driven by a few insane tail observations.

**D. “No RBCS Family” is mildly tail-enriched but not dominated by the tail.**

- `enrichment_ratio` ~ 2.44
- `pct_spend_in_tail` ~ 4.22%

So `'000'` is not the main tail culprit. It is somewhat overrepresented, but the tail is not primarily a “No RBCS” artifact.

**E. Colony Stimulating Factors are meaningfully tail-enriched.**

`enrichment_ratio` ~ 7.23
`pct_spend_in_tail` ~ 12.5%

So this family does have a real upper tail presence and could matter for modeling behavior, depending on what we are predicting and how we evaluate.

**Interpretation of the tail enrichment analysis**

- The top 1% cost-per-service tail is compositionally different from the full dataset.

- Tail spend is highly concentrated in PET- Oncology, which accounts for ~58% of tail spend while being ~2.2% of overall spend (about 26x enrichment).

- Several procedure families are almost entirely tail when present (embolization, arthroplasty, neurostimulator), but they contribute little to overall spend because their total spend is small.

- High-spend baseline families (chemo, IMRT, conventional RT, office E&M) are not tail-dominated, suggesting your primary spend concentration story is robust.

**Implications for modeling**

**A.We should keep the tail but evaluate it explicitly**

Our enrichement analysis makes the tail interpretable rather than "noise." This is why we should not blindly winsorize away signal.

We will
- train our model on `log_stdzd_amt_per_service`
- add an evaluation slice
    - performance on tail rows (`is_top_1pct_stdzd_amt_per_service == True`)
    - performance on non-tail rows (`is_top_1pct_stdzd_amt_per_service == False`)
This tells us if the model is failing exactly where the cost-outlier story lives. 

**B. Consider a two-stage modeling option (only if needed)**

Because PET is so tail-dominant, we might actually get a better model by doing:
1. Classify "tail vs non-tail", then
2. regress cost within each group

This is optional, and will be implemented only if needed. The enrichment analysis justifies this but it is not recommended right away. 

**C. Feature implications**
Because PET is so strong in tail composition, this reinforces that we want strong service-mix controls:
- `rbcs_family_desc` (or its higher-level category) MUST be included 
- `Place_Of_Service` also matters a lot for imaging vs facility pricing patterns. 

## EDA 5. Place-of-service effects within major families

**Plots**

For top 8 families by spend:

1. Box/violin: `stdzd_amt_per_service` by `Place_Of_Srvc`, faceted by `rbcs_family_desc`
2. Stacked bar: share of `services` by `Place_Of_Service` for each `rbcs_family_desc`

**Questions answered**

- How much does site-of-care affect cost per service?
- Is provider variation partly a “site-of-care mix” story?

In [ ]:
(eda_df
 .groupby("rbcs_family_desc", as_index=False)
 .agg(total_stdzd_spend = ("stdzd_spend","sum"), sample_size = ("stdzd_spend", "size"), std_dev_stdzd_spend = ("stdzd_spend", "std"))
 .sort_values("total_stdzd_spend", ascending = False)[:8])

In [ ]:
top_8_rbcs_by_spend = (eda_df
                       .groupby("rbcs_family_desc", as_index=False)
                       .agg(total_stdzd_spend = ("stdzd_spend","sum"))
                       .sort_values("total_stdzd_spend", ascending = False)[:8]["rbcs_family_desc"]
                       .to_list())

eda_top_8 = eda_df.loc[eda_df["rbcs_family_desc"].isin(top_8_rbcs_by_spend),:]

eda_top_8

In [ ]:
g = sns.FacetGrid(eda_top_8, row="rbcs_family_desc", height = 5, aspect=1)

g.map(sns.violinplot, "Place_Of_Srvc", "log_stdzd_amt_per_service")

g.figure.suptitle("Standardized Cost by Place of Service for Top 8 RBCS Families",  y=1.0, fontsize=11)
g.figure.tight_layout() 

plt.show()

Yes, you did this correctly. A FacetGrid of violin plots by rbcs_family_desc, split by Place_Of_Srvc, is a solid way to answer “how much does site-of-care shift the distribution, within major families” while keeping the outcome on a log scale so the tail does not dominate.

Below is how I would read each panel (F vs O), based on the plot you attached.

Interpretation by rbcs_family_desc

Office E&M. New
- F and O are both tightly concentrated (narrow violins), meaning relatively stable “price per service” within this family.
- O shows a bit more heterogeneity (you can see multiple bulges / clumps), suggesting multiple billing patterns or code mixes within outpatient for “new” E&M.

Office E&M. Established
- F and O look very similar in center and spread, with modest tails.
- Takeaway: site-of-care does not dramatically shift the distribution here, at least on the log scale.

Conventional Radiation Treatment
- O is shifted upward vs F and looks a bit wider. That means higher typical log standardized amount per service in outpatient, and possibly more variation.
- Takeaway: this family shows a clear POS effect worth keeping as a modeling signal (or at least controlling for).

No RBCS Family
- This is the most “story-rich” panel.
- F looks multimodal and higher overall (several bulges at higher log values).
- O has a big mass at much lower log values plus a long tail upward.
- This matches what you already found: RBCS 000 is heavily influenced by high-volume injection-style patterns in outpatient, where services explode but implied per-service standardized amount can be low, while facility side can include very different “misc” services with higher per-service costs.

Radiation Treatment Planning
- O is higher than F (the O violin is centered above F and looks more spread).
- Takeaway: another strong POS effect within a radiation-related family.

Intensity Modulated Radiation Therapy
- This looks weird because the violin collapses into a thin line / minimal body.
- That almost always happens when the group has too little variability for KDE to form a distribution, typically due to one of these:
	1.	Very small n in one or both POS groups, sometimes even n=1.
	2.	Near-zero variance (many rows share the same or nearly the same log_stdzd_amt_per_service), so KDE degenerates.
	3.	Heavy discreteness (a few repeated exact values), which can also make a violin look “pinched”.

Confirm which one it is with this quick check:

In [ ]:
tmp = eda_top_8.query("rbcs_family_desc == 'Intensity Modulated Radiation Therapy'")
(tmp.groupby("Place_Of_Srvc")
   .agg(n=("log_stdzd_amt_per_service","size"),
        nunique=("log_stdzd_amt_per_service","nunique"),
        min=("log_stdzd_amt_per_service","min"),
        p25=("log_stdzd_amt_per_service", lambda s: s.quantile(0.25)),
        med=("log_stdzd_amt_per_service","median"),
        p75=("log_stdzd_amt_per_service", lambda s: s.quantile(0.75)),
        max=("log_stdzd_amt_per_service","max"))
)

With those counts, a collapsed violin is not what I would expect. We have plenty of rows in both POS groups (F: 6,929. O: 5,792) and thousands of unique values, so this is not a “small n” or “nunique=1” situation.

What is happening is that the distribution is extremely concentrated in a very narrow band around ~5.80 on the log scale, especially for F:
	•	F: p25 5.803, median 5.807, p75 5.808. That is an interquartile range of about 0.004.
	•	O: also centered around ~5.81 but a bit wider (p25 5.778, p75 5.871), plus a longer upper tail (max 7.34).

For the IMRT we should do boxplot + strip sample because the violin plot collapsed due to
- the underlying KDE bandwidth ends up small relative to the plot scale, and
- you have a tall, thin density spike (lots of mass at nearly the same value)

In [ ]:
sns.boxplot(data=eda_top_8.query("rbcs_family_desc=='Intensity Modulated Radiation Therapy'"),
            x="Place_Of_Srvc", y="log_stdzd_amt_per_service")
sns.stripplot(data=eda_top_8.query("rbcs_family_desc=='Intensity Modulated Radiation Therapy'").sample(2000, random_state=0),
              x="Place_Of_Srvc", y="log_stdzd_amt_per_service", alpha=0.2, size=2)

In [ ]:
# Assign the plot output to a variable (e.g., 'ax') to capture the axes
ax = sns.histplot(
    data=eda_top_8,
    y="rbcs_family_desc",
    hue="Place_Of_Srvc",
    weights="services",
    multiple="fill",
    shrink=0.8,
)
plt.xlabel("Proportion")
plt.ylabel("RBCS Family Description")

# Move the legend
sns.move_legend(
    ax, 
    "upper left", # Anchor point within the legend box
    bbox_to_anchor=(1, 1), # Coordinates (x, y) relative to the plot area (0,0 bottom-left, 1,1 top-right)
    title="Place_Of_Srvc" # Optional: set the legend title
)
plt.title("Place of Service Proportions for Top 8 RBCS Families")
plt.show()

Because we used:
- `weights="services"`
- `multiple="fill"`

each family’s bar is normalized to 1.0, and it answers:

“For this family, what fraction of total services happened in O vs F?”

This is a service-weighted POS mix plot. That is the right way to tell the “site-of-care mix” story.

How to interpret it
- If a family’s bar is almost all O, then any provider-level differences in that family’s spend are mostly happening in office settings.
- If a family’s bar is split or mostly F, then facility settings are a big driver of that family’s utilization and potentially its spend.
- Families like radiation treatment and planning often show more facility share, which is exactly the kind of story this plot is designed to surface.

Let's do the same for the top 1% tail for the `stdzd_amt_per_service`:

In [ ]:
(is_top_1pct_eda_df
 .groupby("rbcs_family_desc", as_index=False)
 .agg(total_stdzd_spend = ("stdzd_spend","sum"))
 .assign(pct_total_stdzd_spend = lambda x: x["total_stdzd_spend"]/x["total_stdzd_spend"].sum())
 .sort_values("total_stdzd_spend", ascending = False)[:8])

In [ ]:
(is_top_1pct_eda_df
 .groupby("rbcs_family_desc", as_index=False)
 .agg(total_services = ("services","sum"))
 .assign(pct_total_services = lambda x: x["total_services"]/x["total_services"].sum())
 .sort_values("total_services", ascending = False)[:8])

## EDA 6. Specialty comparisons (our 5 oncology types)

**Plots**

- Provider counts: n unique `Rndrng_NPI` by `provider_type` and `Year` (table)
- Distribution: `provider_year_stdzd_amt_per_service` by `provider_type` (box)
- Risk score distribution, `bene_avg_risk_score`, by `provider_typ`e (box)
- `services` distribution by `provider_type` (box)

**Questions answered**

- Are specialties fundamentally different in risk, volume, and cost intensity?
- Does it justify including provider_type as a feature?
- Do we need later stratified evaluation by specialty?

In [ ]:
eda_df.groupby(["provider_type", "Year"]).agg(npi_count = ("Rndrng_NPI", "count"))

In [ ]:
g = sns.boxenplot(
    data=eda_df,
    x="provider_year_stdzd_amt_per_service",
    y="provider_type",
    hue="provider_type"
)

g.set_xlabel("Standardized Amount per Service (Provider-Year Grain)")
g.set_ylabel("Provider Type")


plt.show()

In [ ]:
eda_df.loc[eda_df.provider_year_stdzd_amt_per_service>12000,:]

In [ ]:
py = (eda_df[["Rndrng_NPI","Year","provider_type","provider_year_stdzd_amt_per_service"]]
      .drop_duplicates(subset=["Rndrng_NPI","Year"]))

sns.boxenplot(
    data=py,
    x="provider_year_stdzd_amt_per_service",
    y="provider_type"
)
plt.show()

In [ ]:
py.sort_values("provider_year_stdzd_amt_per_service", ascending=False).head(10)

In [ ]:
py.loc[py.provider_year_stdzd_amt_per_service>=13492,:]

In [ ]:
con.sql(
    """
SELECT *
FROM provider_year_features
LIMIT 2
"""
)

In [ ]:
con.sql(
    """
SELECT tot_mdcr_stdzd_amt, tot_srvcs
FROM provider_year_features
WHERE
    Rndrng_NPI=1629222450 AND Year=2022
"""
)

In [ ]:
con.sql(
    """
SELECT *
FROM prov_svc_onco_core_prov_all_rbcs_nppes
WHERE 
    Rndrng_NPI=1629222450 
    AND Year=2022
"""
)

In [ ]:
con.sql(
    """
SELECT *
FROM eda_dataset
WHERE 
    Rndrng_NPI=1629222450 
    AND Year=2022
"""
)

Let's run this to list every HCPCS row for that provider-year, ordered by volume:

In [ ]:
con.sql(
    """
SELECT
  HCPCS_Cd,
  HCPCS_Desc,
  Place_Of_Srvc,
  Tot_Srvcs,
  Avg_Mdcr_Stdzd_Amt,
  (Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) AS stdzd_spend
FROM prov_svc_onco_core_prov_all_rbcs_nppes
WHERE Rndrng_NPI = 1629222450
  AND Year = 2022
ORDER BY Tot_Srvcs DESC;
"""
)

Then sanity check that these HCPCS rows sum to 103:

In [ ]:
con.sql(
    """
SELECT
  SUM(Tot_Srvcs) AS sum_hcpcs_services,
  SUM(Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) AS sum_hcpcs_stdzd_spend
FROM prov_svc_onco_core_prov_all_rbcs_nppes
WHERE Rndrng_NPI = 1629222450
  AND Year = 2022;
"""
)

**There seems to be a mismatch here!**

Let's investigate the following idea:

>CMS suppression (most common)
Many HCPCS-level rows are suppressed when beneficiary counts are small (privacy rule). Provider-year totals can still be reported even if many individual HCPCS lines are suppressed. Result: totals look big, visible HCPCS lines look incomplete.

**Check 1**. Compare provider summary counts vs observable HCPCS counts

In [ ]:
con.sql(
    """
SELECT
  MAX(Tot_Srvcs_1) AS tot_srvcs_summary,
  COUNT(*) AS n_hcpcs_rows_visible,
  SUM(Tot_Srvcs) AS sum_srvcs_visible
FROM prov_svc_onco_core_prov_all_rbcs_nppes
WHERE Rndrng_NPI=1629222450 AND Year=2022;

"""
)

**Check 2**. Is the table inherently “partial” for many providers

Pick a handful of NPIs and compute:

In [ ]:
con.sql(
    """
SELECT
  AVG(sum_srvcs_visible * 1.0 / NULLIF(tot_srvcs_summary,0)) AS avg_visible_share
FROM (
  SELECT
    Rndrng_NPI,
    Year,
    MAX(Tot_Srvcs_1) AS tot_srvcs_summary,
    SUM(Tot_Srvcs) AS sum_srvcs_visible
  FROM prov_svc_onco_core_prov_all_rbcs_nppes
  GROUP BY 1,2
);
"""
)

Let's investigate the following idea

>The table is filtered to an oncology-core or RBCS-mapped subset
The table includes filtering (“onco_core”, “all_rbcs”). If the provider-year totals are coming from the broader provider summary, but the HCPCS lines are only a subset (oncology-related, mapped, or kept after joins), we will also see gaps.

In [ ]:
con.execute(
    """
CREATE OR REPLACE VIEW prov_svc_all_prov_all AS
SELECT
  ps.*,
  p.* EXCLUDE (Rndrng_NPI, Year)
FROM prov_svc_all AS ps
LEFT JOIN prov_all AS p
  ON ps.Rndrng_NPI = p.Rndrng_NPI
 AND ps.Year = p.Year;
""")

In [ ]:
con.sql(
    """
SELECT
  SUM(Tot_Srvcs) AS sum_hcpcs_services,
  SUM(Tot_Srvcs * Avg_Mdcr_Stdzd_Amt) AS sum_hcpcs_stdzd_spend
FROM prov_svc_all_prov_all
WHERE Rndrng_NPI = 1629222450
  AND Year = 2022;
"""
)

In [ ]:
con.sql(
    """
SELECT *
FROM prov_svc_all_prov_all
WHERE 
    Rndrng_NPI=1629222450 
    AND Year=2022
"""
)

***Two additional checks that would make this airtight***

**Check A**: Is the missing share correlated with small benes?

Suppression is usually more common when beneficiary counts are small.

If suppression is the driver, avg_visible_share should be lower in the small-benes buckets.

In [ ]:
con.sql(
    """
WITH py AS (
  SELECT
    Rndrng_NPI,
    Year,
    MAX(Tot_Benes_1) AS tot_benes_summary,
    MAX(Tot_Srvcs_1) AS tot_srvcs_summary,
    SUM(Tot_Srvcs)   AS sum_srvcs_visible
  FROM prov_svc_all_prov_all
  GROUP BY 1, 2
),
bucketed AS (
  SELECT
    CASE
      WHEN tot_benes_summary < 11 THEN '<11'
      WHEN tot_benes_summary < 50 THEN '11-49'
      WHEN tot_benes_summary < 100 THEN '50-99'
      ELSE '100+'
    END AS benes_bucket,
    sum_srvcs_visible * 1.0 / NULLIF(tot_srvcs_summary, 0) AS visible_share
  FROM py
)
SELECT
  benes_bucket,
  AVG(visible_share) AS avg_visible_share,
  APPROX_QUANTILE(visible_share, 0.10) AS p10,
  APPROX_QUANTILE(visible_share, 0.50) AS p50,
  APPROX_QUANTILE(visible_share, 0.90) AS p90,
  COUNT(*) AS n_provider_years
FROM bucketed
GROUP BY 1
ORDER BY
  CASE benes_bucket
    WHEN '<11' THEN 1
    WHEN '11-49' THEN 2
    WHEN '50-99' THEN 3
    ELSE 4
  END;
"""
)

It turns out the hypothesis "Suppression is usually more common when beneficiary counts are small." is false! 

**Check B**: Identify how extreme this provider-year is relative to others

Then check where 35/103 (~0.34) lands in that distribution. My guess is it will be in the “heavily suppressed” tail.

In [ ]:
con.sql(

"""
WITH py AS (
  SELECT
    Rndrng_NPI,
    Year,
    MAX(Tot_Srvcs_1) AS tot_srvcs_summary,
    SUM(Tot_Srvcs) AS sum_srvcs_visible
  FROM prov_svc_all_prov_all
  GROUP BY 1,2
),
ratio AS (
  SELECT
    *,
    sum_srvcs_visible * 1.0 / NULLIF(tot_srvcs_summary,0) AS visible_share
  FROM py
)
SELECT
  MIN(visible_share) AS min_share,
  APPROX_QUANTILE(visible_share, 0.05) AS p05,
  APPROX_QUANTILE(visible_share, 0.50) AS p50,
  APPROX_QUANTILE(visible_share, 0.95) AS p95,
  MAX(visible_share) AS max_share
FROM ratio;
"""
)

Since there is some level of suppression happening (more for the providers with less beneficiaries but more or less consistently across all beneficiary levels), we should check to see how much of this suppression is happening in our `eda_dataset`. 

Let’s now compute the provider-year visible_shares for the rows that actually enter`eda_dataset` after we apply filters (services ≥ 11, RBCS mapped), and see if the distribution changes. 

1) Provider-year visible share for the provider-years that enter eda_dataset

That produces one row per provider-year that actually makes it into `eda_dataset`, with `visible_share_eda`.

In [ ]:
con.sql(
    """
WITH py_eda AS (
  SELECT
    Rndrng_NPI,
    Year,
    SUM(services) AS sum_srvcs_in_eda,
    COUNT(*)      AS n_eda_rows
  FROM eda_dataset
  GROUP BY 1,2
),
py_totals AS (
  SELECT
    Rndrng_NPI,
    Year,
    tot_srvcs AS tot_srvcs_summary,
    tot_benes AS tot_benes_summary
  FROM provider_year_features
)
SELECT
  e.Rndrng_NPI,
  e.Year,
  e.sum_srvcs_in_eda,
  t.tot_srvcs_summary,
  e.sum_srvcs_in_eda * 1.0 / NULLIF(t.tot_srvcs_summary, 0) AS visible_share_eda,
  e.n_eda_rows,
  t.tot_benes_summary
FROM py_eda e
JOIN py_totals t
  ON e.Rndrng_NPI = t.Rndrng_NPI
 AND e.Year = t.Year
LIMIT 60;
"""
)

2) Summarize the distribution and compare to our earlier “HCPCS visible share” baseline

A) Distribution for the eda_dataset provider-years

In [ ]:
con.sql(
    """
WITH py_eda AS (
  SELECT
    Rndrng_NPI,
    Year,
    SUM(services) AS sum_srvcs_in_eda
  FROM eda_dataset
  GROUP BY 1,2
),
joined AS (
  SELECT
    e.Rndrng_NPI,
    e.Year,
    e.sum_srvcs_in_eda,
    p.tot_srvcs AS tot_srvcs_summary,
    e.sum_srvcs_in_eda * 1.0 / NULLIF(p.tot_srvcs, 0) AS visible_share_eda
  FROM py_eda e
  JOIN provider_year_features p
    ON e.Rndrng_NPI = p.Rndrng_NPI
   AND e.Year = p.Year
)
SELECT
  COUNT(*) AS n_provider_years,
  AVG(visible_share_eda) AS avg_share,
  APPROX_QUANTILE(visible_share_eda, 0.05) AS p05,
  APPROX_QUANTILE(visible_share_eda, 0.10) AS p10,
  APPROX_QUANTILE(visible_share_eda, 0.50) AS p50,
  APPROX_QUANTILE(visible_share_eda, 0.90) AS p90,
  APPROX_QUANTILE(visible_share_eda, 0.95) AS p95
FROM joined;
"""
)

B) Side-by-side comparison: “HCPCS visible share” vs “EDA share”, but on the same provider-years

This lets us see how much extra drop is coming from **our filters**, beyond CMS suppression.

In [ ]:
con.sql(
    """
WITH eda_py AS (
  SELECT
    Rndrng_NPI,
    Year,
    SUM(services) AS sum_srvcs_in_eda
  FROM eda_dataset
  GROUP BY 1,2
),
hcpcs_py AS (
  SELECT
    Rndrng_NPI,
    Year,
    MAX(Tot_Srvcs_1) AS tot_srvcs_summary,
    SUM(Tot_Srvcs)   AS sum_srvcs_visible_hcpcs
  FROM prov_svc_all_prov_all
  GROUP BY 1,2
),
joined AS (
  SELECT
    e.Rndrng_NPI,
    e.Year,
    h.tot_srvcs_summary,
    h.sum_srvcs_visible_hcpcs,
    e.sum_srvcs_in_eda,
    h.sum_srvcs_visible_hcpcs * 1.0 / NULLIF(h.tot_srvcs_summary, 0) AS visible_share_hcpcs,
    e.sum_srvcs_in_eda        * 1.0 / NULLIF(h.tot_srvcs_summary, 0) AS visible_share_eda
  FROM eda_py e
  JOIN hcpcs_py h
    ON e.Rndrng_NPI = h.Rndrng_NPI
   AND e.Year = h.Year
)
SELECT
  COUNT(*) AS n_provider_years,
  AVG(visible_share_hcpcs) AS avg_hcpcs_share,
  AVG(visible_share_eda)   AS avg_eda_share,
  APPROX_QUANTILE(visible_share_hcpcs, 0.50) AS p50_hcpcs,
  APPROX_QUANTILE(visible_share_eda,   0.50) AS p50_eda,
  APPROX_QUANTILE(visible_share_hcpcs, 0.90) AS p90_hcpcs,
  APPROX_QUANTILE(visible_share_eda,   0.90) AS p90_eda
FROM joined;
"""
)

1) The `eda_dataset` provider-years have a “pretty complete” picture, on average

From the `visible_share_eda` summary:

- **n = 56,804 provider-years**
- **avg_share = 0.7849**
- **median (p50) = 0.8913**
- **p90 = 0.9923**

Interpretation:

- For the provider-years that make it into `eda_dataset`, we typically observe **most** of their annual services. Median provider-year has about **89%** of services represented.
- But there is still a left tail. **p10 = 0.3469** means 10% of provider-years only have ~35% or less of their annual services represented in `eda_dataset`. So suppression (and/or the service-line inclusion rules) still causes partial visibility for some provider-years.

2) The side-by-side comparison shows us filters are not making visibility worse . If anything, they select “cleaner” provider-years

we got:

- **`avg_hcpcs_share` = 0.7449**
- **`avg_eda_share` = 0.7849**
- **`p50_hcpcs` = 0.8661**
- **`p50_eda` = 0.8913**
- **`p90_hcpcs` = 0.9782**
- **`p90_eda` = 0.9922**

Interpretation:

- If our `eda_dataset` filtering were removing a lot of services beyond CMS suppression, we'd expect **`avg_eda_share` < `avg_hcpcs_share`**.
- We see the opposite. `eda_share` is higher. That means the provider-years that survive into `eda_dataset` tend to be the ones where a larger portion of total services are observable at the service-line level.

**The two nuances not to be missed**

**Nuance A: We are conditioning on “provider-years that enter `eda_dataset`”**

This is selection. We are not saying suppression is gone overall. We are saying:

- “Among provider-years that pass RBCS mapping and services≥11 at the service-line grain, the visible share looks fine.”

That’s good, but it also implies our modeling dataset is biased toward provider-years with:

- higher volumes
- more non-suppressed lines
- more mappable services

That might be acceptable. We just want to be aware of it.

**Nuance B: Our `p95 = 1.0294` is a red flag (shares should not exceed 1)**

A share above 1 can happen because:

- totals and service-line counts are not strictly comparable (different definitions of “services”), or
- duplication/leakage in joins somewhere, or
- rounding plus approximate quantiles, but 1.03 is too high for just rounding.

This does not invalidate the general pattern, but we should sanity check it.

Let's run these two checks:

**Check 1**: how many provider-years have share > 1?

In [ ]:
con.sql(
    """
WITH py_eda AS (
  SELECT Rndrng_NPI, Year, SUM(services) AS sum_srvcs_in_eda
  FROM eda_dataset
  GROUP BY 1,2
),
joined AS (
  SELECT
    e.Rndrng_NPI,
    e.Year,
    e.sum_srvcs_in_eda,
    p.tot_srvcs AS tot_srvcs_summary,
    e.sum_srvcs_in_eda * 1.0 / NULLIF(p.tot_srvcs, 0) AS share
  FROM py_eda e
  JOIN provider_year_features p
    ON e.Rndrng_NPI=p.Rndrng_NPI AND e.Year=p.Year
)
SELECT
  SUM(CASE WHEN share > 1 THEN 1 ELSE 0 END) AS n_gt_1,
  MAX(share) AS max_share
FROM joined;
"""
)

**Check 2**: inspect the worst offenders

In [ ]:
con.sql(
    """
WITH py_eda AS (
  SELECT Rndrng_NPI, Year, SUM(services) AS sum_srvcs_in_eda
  FROM eda_dataset
  GROUP BY 1,2
),
joined AS (
  SELECT
    e.Rndrng_NPI,
    e.Year,
    e.sum_srvcs_in_eda,
    p.tot_srvcs AS tot_srvcs_summary,
    e.sum_srvcs_in_eda * 1.0 / NULLIF(p.tot_srvcs, 0) AS share
  FROM py_eda e
  JOIN provider_year_features p
    ON e.Rndrng_NPI=p.Rndrng_NPI AND e.Year=p.Year
)
SELECT *
FROM joined
WHERE share > 1
ORDER BY share DESC
LIMIT 20;
"""
)

The three key facts:

1. **share > 1 happens a lot**
- 4,198 out of 56,804 provider-years (about **7.4%**) have `sum_srvcs_in_eda` > `tot_srvcs_summary`.
2. **The maximum is exactly 2.0** (and several near 2)
- That pattern (exactly 2x or almost 2x) is a classic signature of **systematic duplication** of service-line rows before we aggregated into `eda_dataset`.
3. **`tot_srvcs_summary` comes from provider summary totals**, while `sum_srvcs_in_eda` comes from summing service-line rows.
- Provider totals are not “double-counted” by joins because we used `MAX(Tot_Srvcs_1)` in the provider-year table.
- Service-line sums can be inflated if the underlying service-line table has duplicate rows for the same “real” HCPCS line.

In [ ]:
con.sql(
    """
SELECT
  HCPCS_Cd,
  Place_Of_Srvc,
  COUNT(*) AS n_rows,
  SUM(Tot_Srvcs) AS sum_srvcs_across_rows,
  MAX(Tot_Srvcs) AS max_srvcs_single_row
FROM prov_svc_all_prov_all
WHERE Rndrng_NPI = 1912087271 AND Year = 2021
GROUP BY 1,2
HAVING COUNT(*) > 1
ORDER BY n_rows DESC, sum_srvcs_across_rows DESC
LIMIT 50;
"""
)

**Quick sanity check:**

we can verify that the problem is “duplication” and not something conceptual by comparing totals:

**Compare provider-year totals vs our EDA service sums**

If `max_diff` is very large (it likely will be for the big offenders), that reinforces this is not rounding. It is duplicated volume.

In [ ]:
con.sql(
    """
WITH eda_py AS (
  SELECT Rndrng_NPI, Year, SUM(services) AS sum_srvcs_in_eda
  FROM eda_dataset
  GROUP BY 1,2
)
SELECT
  COUNT(*) AS n,
  AVG(sum_srvcs_in_eda - p.tot_srvcs) AS avg_diff,
  MAX(sum_srvcs_in_eda - p.tot_srvcs) AS max_diff
FROM eda_py e
JOIN provider_year_features p
  ON e.Rndrng_NPI=p.Rndrng_NPI AND e.Year=p.Year
WHERE sum_srvcs_in_eda > p.tot_srvcs;
"""
)

Given what we just observed, the highest-probability source of duplication is our RBCS join (and much less likely the NPPES join). We should interrogate `prov_svc_onco_core_prov_all_rbcs` first, because that is exactly where a one-to-many mapping can silently replicate service-line rows.

Why the RBCS join is suspicious in our exact code 

```
LEFT JOIN rbcs AS r
  ON p.HCPCS_Cd = r.HCPCS_Cd
```

implicitly assumes **`rbcs.HCPCS_Cd` is unique** (one row per HCPCS code). If it is not unique, then one provider-service HCPCS row becomes 2+ rows after the join. That inflates:

- `SUM(Tot_Srvcs)` when we later aggregate
- `SUM(Avg_Mdcr_Stdzd_Amt * Tot_Srvcs)` (spend numerator)
- and therefore can distort service-line rollups and our later `SUM(services)` in `eda_dataset`

Our earlier “no duplicates” check was done on `prov_svc_all_prov_all`, which is pre-RBCS, so it is totally consistent to see no duplication there but still see duplication later.

**Step 1. Let's prove whether RBCS has duplicates per HCPCS code**

Let's run this first. It tells us whether our join can replicate rows.

In [ ]:
con.sql(
    """
SELECT
  COUNT(*) AS rbcs_rows,
  COUNT(DISTINCT HCPCS_Cd) AS distinct_hcpcs,
  COUNT(*) - COUNT(DISTINCT HCPCS_Cd) AS extra_rows_due_to_duplicates
FROM rbcs;
"""
)

Then let's identify the worst duplicate HCPCS codes:

If this returns anything, our join is one-to-many for those codes.

In [ ]:
con.sql(
    """
SELECT
  HCPCS_Cd,
  COUNT(*) AS n_rbcs_rows
FROM rbcs
GROUP BY 1
HAVING COUNT(*) > 1
ORDER BY n_rbcs_rows DESC
LIMIT 50;
"""
)

**Step 2. Measure row multiplication directly: before vs after RBCS join**

This is the most direct “did the join duplicate rows” check.

Pick the same offender provider-year we flagged (or any). For example: Rndrng_NPI=1912087271, Year=2021.

2A) Row counts

If `post_rbcs` has higher row count and higher `SUM(Tot_Srvcs)`, we have confirmed duplication is introduced by RBCS join.

In [ ]:
con.sql(
    """
SELECT
  'pre_rbcs' AS stage,
  COUNT(*) AS n_rows
FROM prov_svc_onco_core_prov_all
WHERE Rndrng_NPI=1912087271 AND Year=2021

UNION ALL

SELECT
  'post_rbcs' AS stage,
  COUNT(*) AS n_rows
FROM prov_svc_onco_core_prov_all_rbcs
WHERE Rndrng_NPI=1912087271 AND Year=2021;
"""
)

2B) Service totals

In [ ]:
con.sql(
    """
SELECT
  'pre_rbcs' AS stage,
  SUM(Tot_Srvcs) AS sum_tot_srvcs
FROM prov_svc_onco_core_prov_all
WHERE Rndrng_NPI=1912087271 AND Year=2021

UNION ALL

SELECT
  'post_rbcs' AS stage,
  SUM(Tot_Srvcs) AS sum_tot_srvcs
FROM prov_svc_onco_core_prov_all_rbcs
WHERE Rndrng_NPI=1912087271 AND Year=2021;
"""
)

**Step 3. Identify the exact lines getting duplicated after the RBCS join**

Let's do this by comparing the “natural key” of a provider-service line *before* RBCS, then checking how many RBCS matches it gets.

A good key for our provider-service HCPCS data is usually:

(Rndrng_NPI, Year, HCPCS_Cd, Place_Of_Srvc)

(If we want to be extra safe, we can include `HCPCS_Drug_Ind` too.)

In [ ]:
con.sql(
    """
SELECT
  p.HCPCS_Cd,
  p.Place_Of_Srvc,
  COUNT(*) AS n_post_join_rows,
  SUM(p.Tot_Srvcs) AS sum_srvcs_post_join,
  MAX(p.Tot_Srvcs) AS max_srvcs_single
FROM prov_svc_onco_core_prov_all_rbcs AS p
WHERE p.Rndrng_NPI=1912087271 AND p.Year=2021
GROUP BY 1,2
HAVING COUNT(*) > 1
ORDER BY n_post_join_rows DESC, sum_srvcs_post_join DESC
LIMIT 50;
"""
)

If we get results here but not in `prov_svc_all_prov_all`, it is basically a smoking gun.

Then let's inspect one duplicated HCPCS code and see what differs across the duplicated rows (RBCS columns):

In [ ]:
con.sql(
    """
SELECT
  Rndrng_NPI, Year, HCPCS_Cd, Place_Of_Srvc, Tot_Srvcs,
  RBCS_FamNumb,
  RBCS_Id,
  RBCS_Latest_Assignment,
  RBCS_Analysis_Start_Dt,
  RBCS_Analysis_End_Dt,
  Alt_Assignment_Method,
  RBCS_Id_Ever_Reassigned
FROM prov_svc_onco_core_prov_all_rbcs
WHERE Rndrng_NPI=1912087271 AND Year=2021
  AND HCPCS_Cd = '80502'
ORDER BY RBCS_Latest_Assignment DESC, RBCS_Analysis_Start_Dt DESC;
"""
)

## EDA 7. Geography and rurality patterns (without overfitting)

**Plots**

1. Box: stdzd_amt_per_service by state for states with enough rows
2. Box: stdzd_amt_per_service by ruca_bucket
3. Heatmap or grouped bars:
    - average cost by (ruca_bucket, Place_Of_Srvc)

**Questions answered**

- Are costs different in rural vs urban settings?
- Does standardized amount still show residual geography effects?
- Should we keep state and RUCA in the model? (usually yes)

In [ ]:
eda_df.columns

In [ ]:
state_stats = (eda_df.groupby("state")
  .agg(n=("state","size"))
  .reset_index()
)

eligible_states = state_stats.loc[state_stats["n"] >= 1000, "state"]   # pick a threshold
plot_df = eda_df[eda_df["state"].isin(eligible_states)]

# Optional: keep only top 20 by count for readability
top_states = (state_stats[state_stats["state"].isin(eligible_states)]
              .sort_values("n", ascending=False)
              .head(20)["state"])
plot_df = eda_df[eda_df["state"].isin(top_states)]

In [ ]:
order = (plot_df.groupby("state")["log_stdzd_amt_per_service"]
         .median().sort_values(ascending=False).index)

plt.figure(figsize=(12,5))
sns.boxplot(data=plot_df, x="state", y="log_stdzd_amt_per_service", order=order, showfliers=True)
plt.xticks(rotation=45, ha="right")
plt.title("log_stdzd_amt_per_service by state")
plt.tight_layout()

In [ ]:
tmp = eda_df.groupby("ruca_bucket")[["stdzd_spend","services"]].sum()
tmp["svc_weighted_stdzd_amt_per_service"] = tmp["stdzd_spend"] / tmp["services"]
tmp

In [ ]:
g = sns.boxplot(data=eda_df,
                x = "ruca_bucket",
                y="log_stdzd_amt_per_service")
g.set_title("Log Standardized Amount per Service by Ruca Bucket")
g.set_xlabel("Ruca bucket")
g.set_ylabel("Log Standardized amount per service")
plt.show()

In [ ]:
g = eda_df.groupby(["ruca_bucket","Place_Of_Srvc"])[["stdzd_spend","services"]].sum()
heatmap_data = (g["stdzd_spend"] / g["services"]).unstack()

sns.heatmap(heatmap_data, annot=True, fmt=".2f")
plt.title("Service-weighted stdzd_amt_per_service by RUCA and POS")

In [ ]:
count_mat = eda_df.pivot_table(
    index="ruca_bucket", columns="Place_Of_Srvc",
    values="log_stdzd_amt_per_service", aggfunc="size"
)
sns.heatmap(count_mat, annot=True, fmt="g")
plt.title("Row counts by RUCA and POS")

In [ ]:
# Optional masking of categories with less than 200 rows. 

# counts = eda_df.groupby(["ruca_bucket","Place_Of_Srvc"]).size().unstack()
# mask = counts < 200
# sns.heatmap(heatmap_data, annot=True, fmt=".2f", mask=mask)

## EDA 8. Case-mix and feature relationships (risk adjustment proof)

**Plots**

1. Scatter: bene_avg_risk_score vs stdzd_amt_per_service
    - color by Place_Of_Srvc or provider_type
2. Correlation heatmap (numeric features):
    - bene_avg_risk_score, years_since_enumeration, p_*, log_services, log_benes, log_stdzd_amt_per_service

**Questions answered**

- Does risk score relate to costs in plausible ways?
- Do condition features add extra signal beyond risk score?
- Are any features redundant or highly collinear?

In [ ]:
g = sns.relplot(
    data=eda_df,
    x="log_stdzd_amt_per_service",
    y="bene_avg_risk_score",
    kind="scatter",
    row="provider_type"
)
plt.show()

In [ ]:
g = sns.relplot(
    data=eda_df,
    x="log_stdzd_amt_per_service",
    y="bene_avg_risk_score",
    kind="scatter",
    hue="provider_type",
    alpha=0.5
)
plt.show()

In [ ]:
g = sns.relplot(
    data=eda_df,
    x="log_stdzd_amt_per_service",
    y="bene_avg_risk_score",
    kind="scatter",
    row="Place_Of_Srvc"
)
plt.show()

In [ ]:
g = sns.relplot(
    data=eda_df,
    x="log_stdzd_amt_per_service",
    y="bene_avg_risk_score",
    kind="scatter",
    hue="Place_Of_Srvc",
    alpha=0.5
)
plt.show()

In [ ]:
lst = eda_df.columns.to_list()

In [ ]:
[s for s in lst if re.findall("p_.", s)]

In [ ]:
[s for s in lst if re.findall("log_.", s)]

In [ ]:
# 1. Define the variables
cols = [
    'bene_avg_risk_score', 'years_since_enumeration', 'p_cancer6', 
    'p_diabetes', 'p_ckd', 'p_copd', 'p_htn', 
    'log_services', 'log_benes', 'log_stdzd_amt_per_service'
]

# 2. Compute the correlation matrix
# Replace 'df' with our actual DataFrame name
corr_matrix = eda_df[cols].corr()

# 3. Create the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix, 
    annot=True,          # Show correlation values in cells
    cmap='coolwarm',     # Use a diverging color palette (red=pos, blue=neg)
    fmt=".2f",           # Format values to 2 decimal places
    vmin=-1, vmax=1      # Set scale limits for correlation
)
plt.title('Correlation (Pearson) Heatmap of Beneficiary Risk and Service Metrics')
plt.show()

In [ ]:
# 1. Define the variables
cols = [
    'bene_avg_risk_score', 'years_since_enumeration', 'p_cancer6', 
    'p_diabetes', 'p_ckd', 'p_copd', 'p_htn', 
    'log_services', 'log_benes', 'log_stdzd_amt_per_service'
]

# 2. Compute the correlation matrix
# Replace 'df' with our actual DataFrame name
corr_matrix = eda_df[cols].corr(method="spearman")

# 3. Create the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix, 
    annot=True,          # Show correlation values in cells
    cmap='coolwarm',     # Use a diverging color palette (red=pos, blue=neg)
    fmt=".2f",           # Format values to 2 decimal places
    vmin=-1, vmax=1      # Set scale limits for correlation
)
plt.title('Correlation (Spearman) Heatmap of Beneficiary Risk and Service Metrics')
plt.show()

#### **Investigating the standardized cost per service within `eda_df` quantiles:**

In [ ]:
def q1(s): return s.quantile(0.01)
def q5(s): return s.quantile(0.05)
def q90(s): return s.quantile(0.90)
def p95(s): return s.quantile(0.95)
def p99(s): return s.quantile(0.99)

eda_df[["stdzd_amt_per_service", "log_stdzd_amt_per_service"]].agg(
    ["mean", "min", q1, q5, "median", q90, p95, p99, "max"]
)

> *Notice: the `log_stdzd_amt_per_service` jumped from ~6.4 to ~10.6 when we go from 99th percentile to max value. This tells us that the top 1% slice has abnormally high log standardized cost per service. We must further characterize the top 1%.*

**One extra check before finalizing the EDA choice**

Because the `max` is so high, it’s worth confirming whether the top tail is driven by:

- tiny denominators (services=11 but big spend), or
- specific codes / POS, or
- data artifacts.

A quick diagnostic:

In [ ]:
eda_df.assign(
    is_top_1pct = eda_df["stdzd_amt_per_service"] >= eda_df["stdzd_amt_per_service"].quantile(0.99)
).groupby("is_top_1pct").agg(
    n=("stdzd_amt_per_service","size"),
    med_services=("services","median"),
    med_std=("stdzd_amt_per_service","median"),
)

**What this check tells us**

1) The extreme tail is real and it is meaningfully different, not just “a few giant values.”
- Bottom 99%: median `stdzd_amt_per_service` ≈ `70.3`
- Top 1%: median `stdzd_amt_per_service` ≈ `736.7`

That is roughly a 10x jump in the typical value inside the top 1%.

2) The top 1% tends to occur when service volume is smaller.
- Bottom 99%: median services = `142`
- Top 1%: median services = `41`

This pattern is consistent with “high cost per service often shows up in lower-volume lines,” which is plausible in claims data (rare codes, expensive drugs, certain settings). It also means those points can dominate plots on the raw scale even though they are only 1% of rows.

3) It argues against trimming for EDA.
If we trimmed (dropped) the top 1%, we would remove 3,184 rows that likely represent a real, high-cost segment. That can bias descriptive patterns. Winsorization keeps them but stops them from flattening our visualizations.

#### **Let's explicitly do tail analysis**

1. **Tail composition. Who is in the tail?**

Make the tail interpretable by breaking it down by:

- `rbcs_family_desc`
- `Place_Of_Srvc`
- `provider_type
- `state` (or `ruca_bucket`)
- `svc_bucket`

Do it in three ways because they answer different questions:

- **Row share**: “where do tail rows occur”
- **Service-weighted**: “where does tail volume sit”
- **Spend-weighted**: “where does tail money sit”

In [ ]:
tail = eda_df[eda_df["is_top_1pct_stdzd_amt_per_service"]]
base = eda_df

summary = (tail.groupby("rbcs_family_desc", as_index=False)
           .agg(n_rows=("rbcs_family_desc","size"),
                services=("services","sum"),
                spend=("stdzd_spend","sum"))
           .assign(pct_tail_rows=lambda d: d["n_rows"]/len(tail),
                   pct_all_rows=lambda d: d["n_rows"]/len(base),
                   pct_tail_services=lambda d: d["services"]/tail["services"].sum(),
                   pct_tail_spend=lambda d: d["spend"]/tail["stdzd_spend"].sum())
           .sort_values("spend", ascending=False)
           .head(20))

summary

In [ ]:
tail = eda_df[eda_df["is_top_1pct_stdzd_amt_per_service"]]
base = eda_df

summary = (tail.groupby("Place_Of_Srvc", as_index=False)
           .agg(n_rows=("Place_Of_Srvc","size"),
                services=("services","sum"),
                spend=("stdzd_spend","sum"))
           .assign(pct_tail_rows=lambda d: d["n_rows"]/len(tail),
                   pct_all_rows=lambda d: d["n_rows"]/len(base),
                   pct_tail_services=lambda d: d["services"]/tail["services"].sum(),
                   pct_tail_spend=lambda d: d["spend"]/tail["stdzd_spend"].sum())
           .sort_values("spend", ascending=False)
           .head(20))

summary

In [ ]:
tail = eda_df[eda_df["is_top_1pct_stdzd_amt_per_service"]]
base = eda_df

summary = (tail.groupby("provider_type", as_index=False)
           .agg(n_rows=("provider_type","size"),
                services=("services","sum"),
                spend=("stdzd_spend","sum"))
           .assign(pct_tail_rows=lambda d: d["n_rows"]/len(tail),
                   pct_all_rows=lambda d: d["n_rows"]/len(base),
                   pct_tail_services=lambda d: d["services"]/tail["services"].sum(),
                   pct_tail_spend=lambda d: d["spend"]/tail["stdzd_spend"].sum())
           .sort_values("spend", ascending=False)
           .head(20))

summary

In [ ]:
tail = eda_df[eda_df["is_top_1pct_stdzd_amt_per_service"]]
base = eda_df

summary = (tail.groupby("state", as_index=False)
           .agg(n_rows=("state","size"),
                services=("services","sum"),
                spend=("stdzd_spend","sum"))
           .assign(pct_tail_rows=lambda d: d["n_rows"]/len(tail),
                   pct_all_rows=lambda d: d["n_rows"]/len(base),
                   pct_tail_services=lambda d: d["services"]/tail["services"].sum(),
                   pct_tail_spend=lambda d: d["spend"]/tail["stdzd_spend"].sum())
           .sort_values("spend", ascending=False)
           .head(20))

summary

In [ ]:
tail = eda_df[eda_df["is_top_1pct_stdzd_amt_per_service"]]
base = eda_df

summary = (tail.groupby("ruca_bucket", as_index=False)
           .agg(n_rows=("ruca_bucket","size"),
                services=("services","sum"),
                spend=("stdzd_spend","sum"))
           .assign(pct_tail_rows=lambda d: d["n_rows"]/len(tail),
                   pct_all_rows=lambda d: d["n_rows"]/len(base),
                   pct_tail_services=lambda d: d["services"]/tail["services"].sum(),
                   pct_tail_spend=lambda d: d["spend"]/tail["stdzd_spend"].sum())
           .sort_values("spend", ascending=False)
           .head(20))

summary

In [ ]:
tail = eda_df[eda_df["is_top_1pct_stdzd_amt_per_service"]]
base = eda_df

summary = (tail.groupby("svc_bucket", as_index=False)
           .agg(n_rows=("svc_bucket","size"),
                services=("services","sum"),
                spend=("stdzd_spend","sum"))
           .assign(pct_tail_rows=lambda d: d["n_rows"]/len(tail),
                   pct_all_rows=lambda d: d["n_rows"]/len(base),
                   pct_tail_services=lambda d: d["services"]/tail["services"].sum(),
                   pct_tail_spend=lambda d: d["spend"]/tail["stdzd_spend"].sum())
           .sort_values("spend", ascending=False)
           .head(20))

summary

2. **Tail vs non-tail comparisons on key features**

A simple “tail profiling” table is very persuasive:
- median risk score
- median services, benes
- POS mix
- rurality mix

In [ ]:
(eda_df
 .groupby("is_top_1pct_stdzd_amt_per_service", as_index=False)
 .agg(
     median_risk_score = ("bene_avg_risk_score", "median"),
     median_services   = ("services", "median"),
     median_benes      = ("benes", "median")

 ))

In [ ]:
(eda_df
 .groupby(["is_top_1pct_stdzd_amt_per_service","Place_Of_Srvc"], as_index=False)
 .size())

In [ ]:
(eda_df
 .groupby(["is_top_1pct_stdzd_amt_per_service","ruca_bucket"], as_index=False)
 .size())

In [ ]:
[s for s in eda_df.columns if re.findall("p_", s)]

In [ ]:
(eda_df
 .groupby("is_top_1pct_stdzd_amt_per_service", as_index=False)
 .agg(
     cancer     = ("p_cancer6", "mean"),
     diabetes   = ("p_diabetes", "mean"),
     ckd        = ("p_ckd", "mean"),
     copd       = ("p_copd", "mean"),
     htn        = ("p_htn", "mean")
 ))

## **EDA 9. Tail verification + modeling guardrails**

**Plots**

1. One header table. Size and impact of the tail under reliability thresholds: counts and spend share by tail flag x volume bucket
2. Tail composition tables. Who shows up in the tail, by key categories: tail enrichment by `rbcs_family_desc`, `Place_Of_Service`, `provider_type`, `state`, and `ruca_bucket`
3. Outlier table. A small “human inspection” sample that is high-confidence: tail rows with `services >= 50` sorted by either `stdzd_amt_per_service` (extremity) and `stdzd_spend` (impact)

**Questions answered**

- What is the tail made of? Is it real? How big is it?
- Within the tail, are there patterns that indicate data artifacts, low-reliability noise, or a small number of scenarios that should be treated differently (or at least evaluated differently)?

Given our findings (top 1% has median services ~41, and tail spend share is only ~1.7% while services share is ~1.1%), let's make EDA 9 compact and high value.

**A. One header table. Size and impact of the tail under reliability thresholds:**

In [ ]:
(eda_df.
 groupby(["is_top_1pct_stdzd_amt_per_service", "svc_bucket"])
 .agg(
     n_rows = ("services","size"),
     total_services = ("services", "sum"),
     total_spend = ("stdzd_spend","sum"),
     med_cost = ("stdzd_amt_per_service","median"),
     p95_cost = ("stdzd_amt_per_service", lambda s: s.quantile(0.95))
 )
 .assign(
     spend_per_service=lambda d: d["total_spend"] / d["total_services"],
     spend_per_row=lambda d: d["total_spend"] / d["n_rows"],
     pct_services = lambda d: d["total_services"] / d["total_services"].sum(),
     pct_spend = lambda d: d["total_spend"] / d["total_spend"].sum()
 )
 .sort_index())

**What the table is really saying**

1) The “mass of the dataset” is high-volume, low-cost-per-service

- Look at non-tail, 500+:
- pct_services = 0.9728 (97.3% of all services)
- pct_spend = 0.8462 (84.6% of all standardized spend)
- spend_per_service = 14.23
- med_cost = 31.22

So our dataset is dominated by very high-volume rows that are cheap per service. That is not a bug. It is a core reality of the data, and it will strongly influence model learning.

Big takeaway: most rows represent “routine, high-throughput stuff.”

2) Our tail rows are genuinely high cost per service in every volume bucket

Compare spend_per_service tail vs non-tail in each bucket:

- 11–25: 1103 vs 124 (about 9x)
- 25–50: 1168 vs 125 (about 9x)
- 50–100: 1184 vs 108 (about 11x)
- 100–500: 775 vs 73 (about 11x)
- 500+: 732 vs 14 (about 52x)

This is the most important conclusion from this table.

The tail is not “a small denominator artifact.” Even when services are huge (500+), tail rows remain massively more expensive per service.

3) Tail is small in total business terms, but not trivial

From your earlier rollup (and consistent here):

- Tail is about 1.7% of total spend and about 1.1% of total services overall.

Now we can also see where tail spend is coming from:
Tail pct_spend by bucket:

- 11–25: 0.1148%
- 25–50: 0.2203%
- 50–100: 0.3347%
- 100–500: 0.7150% (largest)
- 500+: 0.3461%

So tail spend contribution is mostly 100–500, with meaningful contributions from 50–100 and 500+. Low-volume buckets contribute the least.

4) spend_per_row tells we “business impact per row,” and it explains why 500+ matters

Within each group, spend_per_row rises with services, as expected.

For example, tail rows:

- 11–25: ~17K spend per row
- 25–50: ~42K
- 50–100: ~83K
- 100–500: ~159K
- 500+: ~632K

Interpretation: even though there are only 83 tail rows in 500+, each of those rows represents a lot of dollars. So high-volume tail rows are the “high-impact” ones, even if they are rare.

5) The subtle but important pattern in non-tail

Non-tail spend_per_service drops hard as volume increases:

- 11–25: 124
- 25–50: 125
- 50–100: 108
- 100–500: 73
- 500+: 14

That suggests the high-volume rows tend to be dominated by low-cost-per-service families (we already saw this with RBCS_FamNumb='000' injections and other high-throughput items). This is a dataset composition story, not an arithmetic artifact.

> Across every service-volume bucket, top-1% cost rows have much higher standardized spend per service (roughly 9–11x in low-to-mid volume and over 50x in 500+ volume), indicating the tail reflects real high-intensity service lines rather than low-volume noise

B) Tail composition tables. Who shows up in the tail, by key categories

Let's do these as “tail vs overall” shares. Do not do a million plots.

Table B1: tail enrichment by `rbcs_family_desc` (we already did) We'll keep it. It is our main “tail is concentrated in these families” proof.

Add Table B2–B4: same idea but for POS, specialty, geography

These are cheap and very informative.

In [ ]:
def tail_enrichment_by(cat):
    d = eda_df.copy()
    tot_all = d["stdzd_spend"].sum()
    tot_tail = d.loc[d["is_top_1pct_stdzd_amt_per_service"], "stdzd_spend"].sum()

    all_ = d.groupby(cat, as_index=False).agg(spend_all=("stdzd_spend","sum"))
    tail_ = (d.loc[d["is_top_1pct_stdzd_amt_per_service"]]
               .groupby(cat, as_index=False)
               .agg(spend_tail=("stdzd_spend","sum")))

    out = (all_.merge(tail_, on=cat, how="left")
             .assign(
                 spend_tail=lambda x: x["spend_tail"].fillna(0.0),
                 overall_share=lambda x: x["spend_all"]/tot_all,
                 tail_share=lambda x: np.where(tot_tail>0, x["spend_tail"]/tot_tail, np.nan),
                 enrichment=lambda x: x["tail_share"]/x["overall_share"],
                 pct_spend_in_tail=lambda x: np.where(x["spend_all"]>0, x["spend_tail"]/x["spend_all"], np.nan),
             )
             .sort_values("enrichment", ascending=False))
    return out

These computed columns mean:

- `overall_share` = category’s share of all spend
- `tail_share` = category’s share of tail spend only
- `enrichment` = tail_share / overall_share
    - 1 means “overrepresented in the tail (by spend)”
    - < 1 means “underrepresented in the tail (by spend)”
- `pct_spend_in_tail` = fraction of that category’s spend that sits in the tail

In [ ]:
tail_enrichment_by("Place_Of_Srvc").head(20)

- `O` (Office): `enrichment` 1.056 (slightly overrepresented)
- `F` (Facility): `enrichment` 0.601 (underrepresented)

Takeaways:
- Tail spend is **a bit more office-weighted** than the overall spend mix.
- But this is not a huge effect. It is “directional,” not a dominant story.
- POS is still worth keeping as a feature.
- But POS alone is not explaining the tail.

In [ ]:
tail_enrichment_by("provider_type").head(20)

- `Surgical Oncology`: 5.83x
- `Gynecological Oncology`: 4.82x
- `Radiation Oncology`: 1.39x
- `Hematology-Oncology`: 0.76x
- `Medical Oncology`: 0.75x

Also notice `pct_spend_in_tail`:

- `Surgical Oncology`: 10.1% of its spend is in the tail (very high relative to the global tail share, which is ~1.7%).
- `Gynecological Oncology`: 8.3% in tail.
- `Radiation Oncology`: 2.4% in tail.
- `Hematology-Oncology` and `Medical Oncology`: about 1.3% in tail.

Takeaways

- Tail spend is **strongly enriched** in `Surgical Oncology` and `Gynecological Oncology`.
- `Radiation Oncology` is modestly enriched.
- `Hematology-Oncology` and `Medical Oncology` are underrepresented in the tail relative to their huge baseline spend share.
- This is one of the strongest modeling signals you have found so far.
- It argues for:
    - Keeping `provider_type` in the model (clearly).
    - Considering stratified evaluation by specialty (because the tail behaves differently by specialty).
    - Being cautious with global thresholds for “over-expected.” A single rule may be unfair across specialties.

In [ ]:
tail_enrichment_by("ruca_bucket").head(20)
# (state is fine too, but do top states only)

- `Urban`: enrichment 1.02 (basically neutral)
- `Suburban`: enrichment 0.62
- `Rural`: enrichment 0.29
- `Unknown`: no tail spend

Takeaways

- Tail spend is mostly urban, but that is also because nearly all spend is urban. The enrichment is basically 1.
- Rural and Suburban are underrepresented in tail spend.
- RUCA may matter for baseline differences, but it is not the headline driver of the tail.
- Still worth keeping RUCA in the model, but don’t oversell “rural vs urban” as an explanation for the extreme tail.

In [ ]:
tail_enrichment_by("rbcs_family_desc").head(20)


1) Two different questions are mixed in one table

A. “What drives the tail (in dollars)?”

For that, ignore enrichment at first. Look at tail_share or spend_tail.

From your output, the tail is dominated by a few families:

- `PET- Oncology`
    - tail_share = 0.5808 means ~58.1% of all tail spend is PET.
    - pct_spend_in_tail = 0.4501 means ~45.0% of PET’s total spend lives in the tail.

- `Injection - Colony Stimulating Factors`
    - tail_share = 0.1689 means ~16.9% of tail spend.
    - pct_spend_in_tail = 0.1251 means ~12.5% of its spend lives in the tail.

- `No RBCS Family`
    - tail_share = 0.1021 means ~10.2% of tail spend.

- Next tier (still meaningful dollars):
    - `Intensity Modulated Radiation Therapy`: tail_share = 0.0484 (about 4.8%)
    - `Conventional Radiation Treatment`: tail_share = 0.0348 (about 3.5%)
    - `Chemotherapeutic Agent`: tail_share = 0.0263 (about 2.6%)

Just those are already roughly:
58.1 + 16.9 + 10.2 + 4.8 + 3.5 + 2.6 ≈ 96% of tail spend (ballpark, based on the lines shown)

***That’s your main storyline: the tail is not diffuse. It is concentrated in a handful of families.***

B. “Which families are disproportionately represented in the tail?”

That is what enrichment answers. But enrichment can be misleading when the family is tiny overall.

Example:
- `Vascular Embolization enrichment` ~57, `pct_spend_in_tail` ~0.99
- True statement: “When embolization appears, it is almost always in the tail.”
- But it is only spend_all ≈ 6e5, so it is not a big contributor to overall spend or tail spend.
So for decision-making, treat enrichment like this:
    - High enrichment + big tail spend = important
    - High enrichment + tiny spend = interesting edge case, not a driver

2) What this reveals and whether it matches expectations

Yes, it reveals what we were trying to learn:

- The tail is structurally real and interpretable, not random noise.
- Family mix is the primary driver of tail composition.
- Some high-cost procedure families are “tail heavy” (high enrichment, high `pct_spend_in_tail`).
- Some massive families like `Chemotherapeutic Agent` have huge overall spend, but are underrepresented in the tail (`enrichment` = 0.11, `pct_spend_in_tail` = 0.19%).

That suggests chemo spend is large because of volume and baseline price, not because it generates many extreme per-service outliers.

3) Modeling implications

A. Keep `rbcs_family_desc` (or `RBCS_FamNumb`) as a core feature

The tail behavior is strongly family-specific. Without this feature, the model will blend very different price regimes.

B. Expect interactions

The combination of:

- `rbcs_family_desc` + `provider_type` + `Place_Of_Srvc`
    
    is likely where a lot of the signal lives, especially for the tail-heavy families (PET, colony stimulating factors, certain procedures).
    

C. Consider “two-stage thinking” even if we train one model

We do not have to literally build two models, but conceptually:

- Stage 1: baseline expected cost by family/POS/specialty/risk
- Stage 2: residual variation within family (provider-level effects)

This helps we avoid treating PET-like price structure as “outliers” when they are simply a different regime.

D. Winsorization is still optional for training, helpful for plots

Given our earlier result that the tail is only ~1.7% of spend and ~1.1% of services, we can:

- keep raw values for modeling (especially if using log outcome),
- use clipped plots (p99) for readability in EDA.



4) One improvement to make your narrative bulletproof

Right now we are sorting by enrichment, which is great for “overrepresented.” But for storytelling and prioritization we should present **two companion tables**:

1. Top families by `spend_tail` or `tail_share`
    
    (answers: what actually drives tail dollars)
    
2. Top families by enrichment with a minimum size filter
    
    Example filter: `overall_share` >= 0.001 or `spend_all` >= 1e7
    
    (answers: what is disproportionately tail-heavy among meaningful families)
    

This prevents “tiny but extreme” families from stealing attention.

If we want a simple rule:

- Use enrichment ranking only after filtering to families with at least, say, **0.1% of total spend**.



5) Concrete take-home messages we can write in the notebook

- “The top 1% cost-per-service tail is not random. It is concentrated in specific RBCS families.”
- “PET- Oncology alone accounts for ~58% of tail spend, followed by Colony Stimulating Factors (~17%) and No RBCS Family (~10%).”
- “Several rare procedure families are almost entirely tail spend when they occur, but they contribute little total dollars overall.”
- “This supports keeping service family in the model and evaluating performance within the tail-heavy families.”

In [ ]:
tail_enrichment_by("rbcs_family_desc").sort_values("spend_tail", ascending = False).head(20)

**What drives tail spend (in dollars)**

Our top 20 by `spend_tail` shows a very sharp concentration:

1) The tail is overwhelmingly a PET story, then a small handful of families
- `PET- Oncology`: `tail_share` = 0.5808
    - About 58% of all tail spend comes from PET alone. Also, `pct_spend_in_tail` = 0.4501, meaning 45% of all PET spend sits in the tail. This is both a major tail driver and truly tail-heavy.

- `Injection - Colony Stimulating Factors`: `tail_share` = 0.1689
    - About 17% of tail spend. `pct_spend_in_tail` = 0.1251 (still meaningfully tail-heavy).

- `No RBCS Family`: `tail_share` = 0.1021
    - About 10% of tail spend, but only 4.2% of that family’s spend is tail. So it drives tail dollars mostly because it is big overall, not because it is unusually tail-heavy.

- Next tier:
    - `Intensity Modulated Radiation Therapy`: ~4.8% of tail spend (`pct_spend_in_tail` ~1.0%)
    - `Conventional Radiation`: ~3.5% (`pct_spend_in_tail` ~0.6%)
    - `Chemotherapeutic Agent`: ~2.6% (`pct_spend_in_tail` ~0.2%)

2) A “top 6 families” summary is defensible and strong

Just the first six rows:
- PET (58.1%)
- Colony Stim factors (16.9%)
- No RBCS Family (10.2%)
- IMRT (4.8%)
- Conventional RT (3.5%)
- Chemo agent (2.6%)

Sum: 58.1 + 16.9 + 10.2 + 4.8 + 3.5 + 2.6 = 96.1% of tail spend (using our printed `tail_shares`).

That is a very crisp take-home message.


**The “why” behind the pattern**

A. Two mechanisms are happening at the same time

1. Families that are intrinsically tail-heavy

- PET- Oncology (45% of spend in tail)
- Injection - Clotting Factors (37% of spend in tail, though smaller dollars)
- Mastectomy (22% of spend in tail)
- Vascular Embolization (99% in tail, but tiny dollars)

2.	Families that are huge overall and therefore contribute some tail dollars even if they are not tail-heavy

- Chemo, IMRT, Conventional RT, No RBCS Family
These have low `pct_spend_in_tail`, but because their spend_all is enormous, they still contribute meaningful tail spend.

This distinction is exactly what our columns enable:
- Use tail_share for “tail dollars”
- Use `pct_spend_in_tail` and enrichment for “tail intensity”

**Finally, let's create a compact table for the top 10 by tail spend with just:**

- `rbcs_family_desc`
- `tail_share`
- `pct_spend_in_tail`
- `enrichment`
- `spend_tail` (maybe as $M)

That one table becomes our single slide or single notebook cell that explains the tail.

In [ ]:
tail_tbl = (
    tail_enrichment_by("rbcs_family_desc")
    .sort_values("spend_tail", ascending=False)
    .loc[:, ["rbcs_family_desc", "tail_share", "pct_spend_in_tail", "enrichment", "spend_tail"]]
    .head(10)
    .assign(spend_tail_m=lambda d: d["spend_tail"] / 1e6)
)

tail_tbl

In [ ]:
tail_tbl_round = tail_tbl.assign(
    tail_share=lambda d: d["tail_share"].round(3),
    pct_spend_in_tail=lambda d: d["pct_spend_in_tail"].round(3),
    enrichment=lambda d: d["enrichment"].round(2),
    spend_tail_m=lambda d: d["spend_tail_m"].round(1),
).drop(columns=["spend_tail"])

tail_tbl_round

**What each column means**

- `tail_share`
    - Within the tail only (top 1% by stdzd_amt_per_service), what fraction of tail dollars come from this family.
        - Example: PET- Oncology = 0.581 means 58.1% of all tail spend comes from PET.

- `pct_spend_in_tail`
    - Within this family’s own total spend, what fraction is contributed by tail rows.
        - Example: Vascular Embolization = 0.991 means 99.1% of embolization spend is in tail rows (for our definition of tail).

- `enrichment`
    - How “overrepresented” the family is in the tail relative to overall.
    - It is essentially: `(tail_share / overall_share)`.
        - Example: PET- Oncology enrichment 26.00 means PET makes up a much larger share of tail spend than it does of total spend.

- `spend_tail_m`
    - Tail spend for that family, in $ millions.

All of these are internally consistent with the earlier outputs we showed.

**What our specific results say**

1) The tail is not broad. It is dominated by a few families.

If we sum the `tail_share` in our top 10:

0.581 + 0.169 + 0.102 + 0.048 + 0.035 + 0.026 + 0.016 + 0.013 + 0.004 + 0.002
= 0.996 (about 99.6% of tail spend)

So basically, the top 10 families explain almost the entire tail. That is exactly the kind of clean narrative we want.

2) PET- Oncology is the main tail story.
- $152.3M of tail spend.
- 58.1% of tail spend.
- Enrichment 26x means it is massively overrepresented in tail dollars.

This strongly suggests our tail is not “random weirdness.” It is a real phenomenon concentrated in certain service families.

3) “High enrichment” and “high tail spend” are different concepts. You have both.
- High tail spend families: PET, Colony Stimulating Factors, No RBCS Family, IMRT, Conventional Radiation.
- Highest enrichment families (even if small dollars): Vascular Embolization, Injection Clotting Factors, PET, Mastectomy.

Those tiny-dollar but huge-enrichment families are still important because they tell we, “when this family appears, it is often expensive per service.”

4) Some large-spend families are actually underrepresented in the tail.
- Chemotherapeutic Agent: enrichment 0.11, pct_spend_in_tail 0.002.
That means chemo drives lots of overall spend, but it is not usually in our extreme cost-per-service tail. It is a different cost story.
- IMRT and Conventional Radiation also have enrichment < 1, meaning they are not tail-heavy relative to their overall footprint, even though they contribute meaningful tail dollars.

**Modeling implications**

Keep the tail. Do not winsorize for training.

Your table screams “structured tail,” not noise.
- Tail rows are systematically linked to specific families and POS patterns.
- If we clip them for training, we will erase real signal and bias the model toward “average-only” behavior.

But for visuals and simple comparisons, we can use a plot-only clip

Two-track approach:
- Modeling outcome: `log_stdzd_amt_per_service` (no clipping)
- EDA plots: optionally show a second version clipped at p99 or p99.5 for readability, clearly labeled “clipped for display only.”

Consider adding a simple “tail-prone family” indicator for diagnostics

Not necessarily as a final feature, but as a quick check:
- a flag like `is_tail_enriched_family` for the top few enrichment families (PET, clotting factors, etc.)

This helps we confirm that residuals and errors behave differently in these families.

One sentence narrative:

> “The cost-per-service tail is highly structured: the top 10 RBCS families explain ~99.6% of tail spend, led by PET-Oncology alone at ~$152M (58% of tail dollars), with several families showing extreme tail concentration (e.g., embolization has ~99% of its spend in tail rows).”

## EDA 10. Modeling readiness summary (what EDA implies)

This section locks the modeling decisions implied by EDA. It serves as a contract for what will be trained, how it will be evaluated, and how “flagging” will be interpreted.

**Dataset grain (unit of analysis):** provider-year-RBCS family-place of service.

We explicitly separate:
- **Training inclusion rules** (coverage + stability)
- **Post-model flagging rules** (higher confidence, higher volume)

10.1 Objective, unit of prediction, and target

**Goal:** predict expected cost intensity per service for each provider-year-family-POS slice, adjusting for case-mix and context.

**Unit of prediction (row grain):**
- Provider (`Rndrng_NPI`)
- Year (`Year`)
- Service family (`RBCS_FamNumb` / `rbcs_family_desc`)
- Place of service (`Place_Of_Srvc`)

**Final modeling target:**
- `y = log1p(stdzd_amt_per_service)` (i.e., `log_stdzd_amt_per_service`)

**Why log target:**
- Raw `stdzd_amt_per_service` is heavy-tailed. Log transform stabilizes training while preserving rank and relative differences.

In [ ]:
import numpy as np
import pandas as pd

target_decision = pd.DataFrame({
    "Decision": [
        "Outcome for reporting",
        "Modeling target",
        "Justification"
    ],
    "Value": [
        "stdzd_amt_per_service (raw, $/service)",
        "log1p(stdzd_amt_per_service) (stored as log_stdzd_amt_per_service)",
        "Heavy-tailed raw outcome. Log reduces tail dominance and improves stability."
    ]
})
target_decision

What it says
- Outcome for reporting: stdzd_amt_per_service (raw dollars per service)
- Modeling target: log1p(stdzd_amt_per_service) which you already store as log_stdzd_amt_per_service
- Justification: raw costs are heavy-tailed, log makes learning stable

How to interpret
- You are explicitly separating:
- what stakeholders want to see and understand (raw dollars), from
- what the model needs to learn well (log dollars).
- This is a strong and defensible stance because your EDA already proved:
- max is huge (tens of thousands),
- p99 is hundreds,
- median is ~70.

Modeling implication
- Train the model on log_stdzd_amt_per_service.
- When you report results, convert back to dollars carefully:
- predicted log-cost -> expm1(pred) gives predicted $/service.
- residuals can be measured on log scale (stable) and optionally translated back to $ for interpretation.


10.2 Final training inclusion rules (EDA-aligned)

**Training inclusion filter:**
- `services >= 11`
- `stdzd_amt_per_service` is not null
- `stdzd_amt_per_service >= 0`

These rules prioritize reliability (avoid tiny denominators) and avoid invalid cost values.

In [ ]:
# Define training inclusion mask explicitly (even if eda_df is already filtered)
train_mask = (
    (eda_df["services"] >= 11) &
    (eda_df["stdzd_amt_per_service"].notna()) &
    (eda_df["stdzd_amt_per_service"] >= 0)
)

train_filter_summary = pd.DataFrame({
    "Metric": [
        "Rows in eda_df",
        "Rows meeting training filter",
        "Share kept",
        "Unique NPIs (all)",
        "Unique NPIs (kept)",
        "Years present (all)",
        "Years present (kept)",
    ],
    "Value": [
        len(eda_df),
        int(train_mask.sum()),
        float(train_mask.mean()),
        eda_df["Rndrng_NPI"].nunique(),
        eda_df.loc[train_mask, "Rndrng_NPI"].nunique(),
        sorted(eda_df["Year"].unique().tolist()),
        sorted(eda_df.loc[train_mask, "Year"].unique().tolist()),
    ]
})

train_filter_summary

The output
- Rows in eda_df: 318,322
- Rows meeting training filter: 318,322
- Share kept: 1.0
- Unique NPIs: 20,919
- Years: [2021, 2022, 2023]

How to interpret
- This confirms a key fact: your eda_df is already “training-ready” with respect to your inclusion rules.
- Your SQL filter WHERE ps.services >= 11 AND ... already enforced the train mask, so the explicit train_mask does not drop anything.

Modeling implication
- No additional inclusion filtering is needed in the modeling notebook.
- That is good because it prevents “silent changes” between EDA and modeling.

Narrative implication
- You can state in Notebook 10:
“The modeling dataset equals the EDA dataset. No additional exclusions are introduced at modeling time.”


10.3 Tail policy (keep tail, do not winsorize for training)

**Key EDA finding:** the top 1% of `stdzd_amt_per_service` is not a handful of errors. It contains many rows and meaningful service volumes.

**Policy:**
- Do **not** winsorize labels for training.
- Use `log_stdzd_amt_per_service` as the modeling target to handle skew.
- Keep `is_top_1pct_stdzd_amt_per_service` as a diagnostic flag for slicing and interpretation.

**Visualization note:**
- Clipping at p99 is allowed **only for readability in plots**, not for training labels.

**10.3A: Tail footprint (rows, spend, services)**

In [ ]:
tail_footprint = (
    eda_df
    .assign(is_tail=eda_df["is_top_1pct_stdzd_amt_per_service"].astype(bool))
    .groupby("is_tail")
    .agg(
        n_rows=("services", "size"),
        total_services=("services", "sum"),
        total_spend=("stdzd_spend", "sum"),
        med_services=("services", "median"),
        med_cost=("stdzd_amt_per_service", "median"),
    )
    .assign(
        pct_rows=lambda d: d["n_rows"] / d["n_rows"].sum(),
        pct_services=lambda d: d["total_services"] / d["total_services"].sum(),
        pct_spend=lambda d: d["total_spend"] / d["total_spend"].sum(),
    )
)

tail_footprint

Your output (key numbers)
- Tail flag is top 1% of stdzd_amt_per_service by construction.
- Non-tail:
    - n_rows = 315,138 (99.0%)
    - total_services ≈ 925,832,559.3
    - total_spend ≈ 1.488e10
    - median services = 142
    - median cost = ~70.3
    - share of spend = 98.27%
- Tail:
    - n_rows = 3,184 (1.0%)
    - total_services = 298,384
    - total_spend = 2.621e8
    - median services = 41
    - median cost = ~736.7
    - share of spend = 1.73%
    - share of services = 0.032%

How to interpret
This table answers three “so what?” questions:

A) Is the tail large in rows?
- It’s exactly 1% (3,184 rows). So it is not a tiny handful, but it is still a small slice.

B) Is the tail large in volume?
- No. Tail is only 0.032% of services.
- That is important: these are high cost per service, not high frequency.

C) Is the tail a major share of dollars?
- Surprisingly, also not huge: 1.73% of total spend.
- That means your dataset’s total spend is driven by the non-tail bulk, even though tail costs look extreme per service.

Modeling implication
- This strongly supports: no winsorization of labels needed.
- The tail is unlikely to dominate training loss when using log target, because:
    - it’s only 1% of rows,
    - it’s tiny in service volume,
    - log compresses extremes further.

Evaluation implication
- You should still slice metrics by tail vs non-tail, because the model might behave differently there.
- But tail is not your “main money story.” It is mainly a “rare high unit cost” story.


**10.3B: Tail explanation table (top 10 families by tail spend)**

In [ ]:
tail_tbl = (
    tail_enrichment_by("rbcs_family_desc")
    .sort_values("spend_tail", ascending=False)
    .loc[:, ["rbcs_family_desc", "tail_share", "pct_spend_in_tail", "enrichment", "spend_tail"]]
    .head(10)
    .assign(spend_tail_m=lambda d: d["spend_tail"] / 1e6)
    .assign(
        tail_share=lambda d: d["tail_share"].round(3),
        pct_spend_in_tail=lambda d: d["pct_spend_in_tail"].round(3),
        enrichment=lambda d: d["enrichment"].round(2),
        spend_tail_m=lambda d: d["spend_tail_m"].round(1),
    )
    .drop(columns=["spend_tail"])
)

tail_tbl

This is our “one slide” tail explanation.

Let’s interpret each column precisely:

-  `tail_share`:

“Of all dollars inside the tail, how much is contributed by this family?”

Example:
- PET-Oncology `tail_share` = 0.581
Means 58.1% of all tail spend is PET-Oncology.

This is a within-tail composition measure.

-  `pct_spend_in_tail`:

“Of this family’s total spend overall, what fraction lies in the tail?”

Example:
- Vascular Embolization `pct_spend_in_tail` = 0.991
Means 99.1% of all Vascular Embolization spend occurs in tail rows.

This is a within-family concentration measure.

- `enrichment`:

“How overrepresented is this category inside the tail compared to its overall prevalence?”

Formally:
- `enrichment` = tail_share / overall_share

Example:
- PET-Oncology `enrichment` = 26
Means PET-Oncology is 26x more concentrated inside the tail than expected if tail mirrored overall spend mix.

- `spend_tail_m`:

Raw dollars in the tail for that family in millions.

Example:
- PET-Oncology `spend_tail_m` = 152.3
Means PET-Oncology accounts for about $152.3M of tail spend.

What the `tail_tbl` tells you
- Tail spend is overwhelmingly driven by a few families, especially:
- PET-Oncology
- Colony Stimulating Factors
- No RBCS Family (misc bucket)
- Some families have extreme pct_spend_in_tail (like Vascular Embolization) but small dollars overall.

Modeling implication
- The tail is not “random noise.” It is structured by service type.
- That is good news. It means categorical features like `rbcs_family_desc` and `Place_Of_Srvc` can help the model learn expected ranges.


10.4 Post-model flagging policy (separate from training)

Training includes rows with `services >= 11`.

For any downstream “flagging” use case (over-expected cost), we should apply stricter volume thresholds to ensure stability:
- **Candidate flagging threshold:** `services >= 50` (primary)
- **High-confidence slice:** `services >= 100` (sensitivity)

We will report model performance and flagging behavior across service-volume buckets.

**10.4A: Table A1 (tail flag x svc_bucket)**

In [ ]:
table_a1 = (
    eda_df
    .groupby(["is_top_1pct_stdzd_amt_per_service", "svc_bucket"], dropna=False, observed=True)
    .agg(
        n_rows=("services", "size"),
        total_services=("services", "sum"),
        total_spend=("stdzd_spend", "sum"),
        med_cost=("stdzd_amt_per_service", "median"),
        p95_cost=("stdzd_amt_per_service", lambda s: s.quantile(0.95)),
    )
    .assign(
        spend_per_service=lambda d: d["total_spend"] / d["total_services"],
        spend_per_row=lambda d: d["total_spend"] / d["n_rows"],
    )
    .assign(
        pct_services=lambda d: d["total_services"] / d["total_services"].sum(),
        pct_spend=lambda d: d["total_spend"] / d["total_spend"].sum(),
    )
    .sort_index()
)

table_a1

This table explains how volume interacts with tail membership.

Key patterns to notice

A) Most services are in non-tail 500+
- Non-tail 500+:
    - total_services ≈ 900,902,470
    - pct_services ≈ 0.973 (97.3% of all services)
    - pct_spend ≈ 0.846 (84.6% of all spend)
    - spend_per_service ≈ 14.23

Interpretation:
- The dataset is dominated by very high volume slices with low $/service.

B) Tail rows have much higher spend per service in every bucket

- Compare spend per service:
    - Non-tail 11-25: ~124
    - Tail 11-25: ~1103
    - Non-tail 100-500: ~73
    - Tail 100-500: ~775
    - Non-tail 500+: ~14
    - Tail 500+: ~732

Interpretation:
- Tail is fundamentally a high unit cost regime regardless of volume bucket.

C) Tail spend is concentrated in mid-volume buckets, not the ultra-high bucket
- Look at tail rows:
    - Tail 500+: only 83 rows
    - Tail 100-500: 681 rows
    - Tail 11-25: 1016 rows

Interpretation:
- The tail is mostly happening in low to mid volume slices, not the massive 500+ slices.

Modeling implication
- This supports your split policy:
    - train on services >= 11 to have enough data
    - flagging at services >= 50 or >= 100 for stability
- It also suggests:
    - if you later do any “confidence scoring,” services volume should be part of it.


**10.4B: Flagging thresholds slice sizes**

In [ ]:
flag_slices = pd.DataFrame({
    "Slice": ["services >= 11 (train)", "services >= 50 (flagging)", "services >= 100 (high confidence)"],
    "Rows": [
        int((eda_df["services"] >= 11).sum()),
        int((eda_df["services"] >= 50).sum()),
        int((eda_df["services"] >= 100).sum()),
    ],
    "Share of rows": [
        float((eda_df["services"] >= 11).mean()),
        float((eda_df["services"] >= 50).mean()),
        float((eda_df["services"] >= 100).mean()),
    ],
    "Share of spend": [
        float(eda_df.loc[eda_df["services"] >= 11, "stdzd_spend"].sum() / eda_df["stdzd_spend"].sum()),
        float(eda_df.loc[eda_df["services"] >= 50, "stdzd_spend"].sum() / eda_df["stdzd_spend"].sum()),
        float(eda_df.loc[eda_df["services"] >= 100, "stdzd_spend"].sum() / eda_df["stdzd_spend"].sum()),
    ],
})
flag_slices

Your output
- services >= 50 includes:
    - 70.0% of rows
    - 97.6% of spend
- services >= 100 includes:
    - 55.7% of rows
    - 95.0% of spend

How to interpret
- Moving from 11 to 50 removes 30% of rows but almost no spend.
- That is a strong argument that “flagging” should happen at >= 50:
    - fewer noisy small slices,
    - you still cover almost all dollars.

Modeling implication
- Keep training on 11.
- Use >= 50 for “provider-facing” flags.
- Use >= 100 for “high confidence” flags.

This is a clean and defensible tiering.


10.5 Final feature list (categorical vs numeric)

We define the final candidate feature list based on EDA:
- categorical context (service family, POS, provider type, geography, rurality)
- numeric case-mix and intensity (risk score, log volume, comorbidity proportions, experience)

We also document excluded columns (leakage, targets, or derived totals not appropriate for the intended modeling objective).

**10.5A: Define final feature lists**

In [ ]:
# Categorical features (to encode)
cat_features = [
    "rbcs_family_desc",    # or use "RBCS_FamNumb" instead, but pick one consistently
    "Place_Of_Srvc",
    "provider_type",
    "state",
    "ruca_bucket",
]

# Numeric features
num_features = [
    "bene_avg_risk_score",
    "years_since_enumeration",
    "log_services",
    "log_benes",
    "p_cancer6", "p_diabetes", "p_ckd", "p_copd", "p_htn",
]

# Final target
target_col = "log_stdzd_amt_per_service"

# Exclusions (documented)
excluded = [
    # raw cost outcomes besides target
    "stdzd_amt_per_service", "allowed_amt_per_service", "payment_amt_per_service", "submitted_charge_per_service",
    # totals derived from outcomes (spend columns)
    "stdzd_spend", "allowed_spend", "payment_spend", "submitted_spend",
    # flags/buckets used for slicing, not for training features
    "is_top_1pct_stdzd_amt_per_service", "svc_bucket", "services_bins", "services_custom", "services_custom2",
    # provider-year totals (often avoided to prevent scale leakage; can revisit intentionally later)
    "tot_mdcr_stdzd_amt",
]

features_table = pd.DataFrame({
    "Type": (["categorical"] * len(cat_features)) + (["numeric"] * len(num_features)) + (["target"] * 1),
    "Column": cat_features + num_features + [target_col]
})

features_table

This is your intended modeling feature set.

Categorical features
- `rbcs_family_desc`
- `Place_Of_Srvc`
- `provider_type`
- `state`
- `ruca_bucket`

Interpretation:
- These explain systematic price differences due to:
    - what service is being delivered,
    - where it is delivered,
    - what specialty is delivering it,
    - geography and rurality.

Numeric features
- risk and experience:
    - `bene_avg_risk_score`
    - `years_since_enumeration`
- exposure/intensity controls (log):
    - `log_services`
    - `log_benes`
- case mix proportions:
    - `p_cancer6`, `p_diabetes`, `p_ckd`, `p_copd`, `p_htn`

Interpretation:
- This is a classic “risk adjustment + context” set.
- You are not leaking the target because you excluded spend totals and other cost measures.

Modeling implication
- You should decide one thing now:
    - `rbcs_family_desc` vs `RBCS_FamNumb`
    - Pick one and use consistently (ID is often cleaner, desc is often fine too).

**10.5B: Quick availability check**

In [ ]:
missing_cols = [c for c in (cat_features + num_features + [target_col]) if c not in eda_df.columns]
missing_cols

missing_cols

Output: []

Interpretation
- Everything you plan to model exists in your dataframe.
- This prevents “modeling notebook surprises.”


10.6 Split plan and leakage checks

**Planned temporal split:**
- Train: 2021–2022
- Test: 2023

We will also quantify provider overlap between train and test:
- How many NPIs appear in both periods (seen providers)?
- How many NPIs are test-only (cold-start providers)?

**10.6A: Create split masks**

In [ ]:
train_years = [2021, 2022]
test_years = [2023]

mask_train = eda_df["Year"].isin(train_years)
mask_test = eda_df["Year"].isin(test_years)

split_counts = pd.DataFrame({
    "Split": ["train", "test"],
    "Years": [train_years, test_years],
    "Rows": [int(mask_train.sum()), int(mask_test.sum())],
    "Unique NPIs": [eda_df.loc[mask_train, "Rndrng_NPI"].nunique(), eda_df.loc[mask_test, "Rndrng_NPI"].nunique()],
    "Spend share": [
        float(eda_df.loc[mask_train, "stdzd_spend"].sum() / eda_df["stdzd_spend"].sum()),
        float(eda_df.loc[mask_test, "stdzd_spend"].sum() / eda_df["stdzd_spend"].sum()),
    ],
})
split_counts

Your output
- Train:
    - 213,296 rows
    - 19,838 NPIs
    - 66.6% spend
- Test:
    - 105,026 rows
    - 19,226 NPIs
    - 33.4% spend

Interpretation
- You have a healthy split. About one-third of dollars are in the test year.
- The train and test have similar provider counts, which is good for generalization tests.

Modeling implication
- This is a realistic production-like test: learn patterns from earlier years, apply to the next year.


**10.6B: Provider overlap (leakage / generalization check)**

In [ ]:
npi_train = set(eda_df.loc[mask_train, "Rndrng_NPI"].unique().tolist())
npi_test = set(eda_df.loc[mask_test, "Rndrng_NPI"].unique().tolist())

overlap = npi_train.intersection(npi_test)
test_only = npi_test - npi_train

provider_overlap_tbl = pd.DataFrame({
    "Metric": ["Train NPIs", "Test NPIs", "Overlap NPIs", "Test-only NPIs"],
    "Value": [len(npi_train), len(npi_test), len(overlap), len(test_only)],
    "Share of test NPIs": [
        np.nan,
        1.0,
        len(overlap)/len(npi_test) if len(npi_test) else np.nan,
        len(test_only)/len(npi_test) if len(npi_test) else np.nan,
    ]
})
provider_overlap_tbl

Your output
- Test NPIs: 19,226
- Overlap with train: 18,145 (94.38%)
- Test-only: 1,081 (5.62%)

Interpretation
- Most test providers were seen in train. That means your evaluation is mostly:
    - “new year for known providers” (easier)
- But you still have a meaningful cold-start set:
    - 1,081 providers

Modeling implication
- You should report performance separately for:
    - seen providers (in train)
    - unseen providers (test-only)

Because those are different deployment realities.


**10.7 Planned evaluation metrics and slices**

**Primary metrics (on log target):**
- MAE on `log_stdzd_amt_per_service`
- RMSE on `log_stdzd_amt_per_service`

**Reporting slices (diagnostics + fairness + stability):**
- by `provider_type`
- by `Place_Of_Srvc`
- by `ruca_bucket`
- by service-volume bucket (`svc_bucket`)
- by tail flag (`is_top_1pct_stdzd_amt_per_service`, diagnostic only)

We will compute these slices on the **test year** (2023) and optionally compare against train.

**10.7A: Planned slice keys table**

In [ ]:
eval_plan = pd.DataFrame({
    "Category": [
        "Primary metrics",
        "Target scale",
        "Core slices (report)",
        "Stability slices (flagging)",
        "Tail diagnostic"
    ],
    "Plan": [
        "MAE, RMSE",
        "log_stdzd_amt_per_service (log1p)",
        "provider_type, Place_Of_Srvc, ruca_bucket, state",
        "svc_bucket and services>=50 / >=100 subsets",
        "Compare tail vs non-tail behavior without changing labels"
    ]
})
eval_plan

This is the written plan. The main implication is the slicing discipline.

Modeling implication
- Even a good average MAE can hide:
    - a big error for one specialty,
    - or systematic underprediction in facility setting.
- Slices are not “nice to have.” They are how you prove risk adjustment is working.


**10.8 Modeling readiness summary table (final contract)**

This final table summarizes the modeling choices implied by EDA and tail analysis.

In [ ]:
readiness_contract = pd.DataFrame({
    "Decision Area": [
        "Dataset grain",
        "Final modeling target",
        "Training inclusion",
        "Tail handling",
        "Visualization-only clipping",
        "Post-model flagging threshold",
        "Features (categorical)",
        "Features (numeric)",
        "Split plan",
        "Evaluation slices"
    ],
    "Final Choice": [
        "provider-year-RBCS family-place of service",
        "log_stdzd_amt_per_service = log1p(stdzd_amt_per_service)",
        "services >= 11; stdzd_amt_per_service not null and >= 0",
        "Keep tail; do not winsorize labels; use tail flag for diagnostics",
        "Allow p99 clipping for readability in plots only",
        "Flagging candidates evaluated on services >= 50 (and >= 100 sensitivity)",
        ", ".join(cat_features),
        ", ".join(num_features),
        "Train 2021–2022, Test 2023",
        "provider_type, Place_Of_Srvc, ruca_bucket, svc_bucket, tail flag (diagnostic)"
    ],
    "Why defensible": [
        "Matches EDA grain and planned use case",
        "Controls heavy tail while preserving signal",
        "Reliability threshold reduces denominator noise",
        "Tail appears real and interpretable (not pure error). Log target handles skew",
        "Prevents misleading plots without altering training distribution",
        "Reduces false positives from low-volume instability",
        "Captures key context (service, POS, specialty, geography, rurality)",
        "Captures case-mix + intensity + experience + comorbidity composition",
        "Temporal generalization is the real-world test",
        "Ensures interpretability and stability across key segments"
    ]
})

readiness_contract

This is your “final decision table.” It ties together all EDA conclusions into a fixed plan.

Interpretation
- This is the single best artifact to prevent scope drift.
- It will make your writeup defensible.

**The data frame for modeling**

In [ ]:
model_cols = cat_features + num_features + [target_col, "Year", "Rndrng_NPI", "services", "stdzd_amt_per_service"]

model_df = eda_df.loc[train_mask, model_cols].copy()

train_df = model_df[model_df["Year"].isin(train_years)].copy()
test_df = model_df[model_df["Year"].isin(test_years)].copy()

train_df.shape, test_df.shape

Output
- Train: (213,296, 19)
- Test: (105,026, 19)

Interpretation
- 19 columns includes:
    - features + target + Year + NPI + services + raw cost
- This is a manageable size.

Modeling implication
- You are ready to encode categoricals and fit a baseline model.


**Missingness of features used in modeling**

In [ ]:
feature_missing = (
    model_df[cat_features + num_features + [target_col]]
    .isna()
    .mean()
    .mul(100)
    .sort_values(ascending=False)
    .rename("pct_missing")
    .reset_index()
    .rename(columns={"index": "column"})
)
feature_missing

This is the one you should pay attention to before training.

You have missingness in:
- `p_copd`: 4.59%
- `p_ckd`: 2.05%
- `p_diabetes`: 1.15%
- `p_cancer6`: 0.86%
- `years_since_enumeration`: 0.45%
- `p_htn`: 0.05%
Everything else: 0%

Interpret each variable’s missingness and what it implies

- `p_copd` (4.59% missing)
    - This is the highest missingness among your features.
    - Likely causes:
        - certain provider-year records lack the COPD percentage due to suppression, reporting rules, or merge gaps.
    - Modeling risk:
        - dropping rows would throw away ~4.6% of your data for one feature.
    - Recommended handling:
        - impute missing with a neutral value (commonly 0) plus a missingness indicator, or
        - impute with median and add indicator.
    - Why indicator matters:
    - missingness might correlate with provider type or geography, so the “missing” itself can carry signal.

- p_ckd (2.05% missing)
    - Similar logic, lower magnitude.
    - You should handle it the same way as p_copd for consistency.

- `p_diabetes` (1.15% missing)
    - Small but non-trivial.
    - Same treatment.

- `p_cancer6` (0.86% missing)
    - Small. Same treatment.
    - This one is especially sensitive conceptually in oncology, so do not silently drop rows.

- `years_since_enumeration` (0.45% missing)
    - This is “provider experience proxy.”
    - Missingness likely means:
        - NPI enumeration date missing upstream, or mapping failed.
    - Modeling risk:
        - leaving it missing can break some models.
    - Handling:
        - impute median and add missingness flag is the safest.

- `p_htn` (0.05% missing)
    - Very small. Still handle systematically (same imputation pattern).
    - Consistency matters. You do not want special-case logic for one feature.

- All categoricals have 0% missing
    - That is excellent. It means your slicing features are complete.
    - Especially important for:
        - `provider_type`, `Place_Of_Srvc`, `ruca_bucket`.

- `bene_avg_risk_score` has 0% missing
    - This is great because it is typically your primary adjustment feature.

Modeling implication
- You need a missingness strategy as part of Notebook 10 or the first modeling notebook.
- The best practice approach here is:

1.	For each numeric feature with missing:

•	create `is_missing_<feature>` indicator

2.	Impute missing values:

•	either 0 (for percentage fields) or median

3.	Keep the indicator in the model

This preserves rows, avoids bias from dropping, and allows missingness patterns to be learned.
